In [3]:
# finetune_mindflow_ulysses.py 初版初级匹配意识流
import os
import json
import time
import torch
import random
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForCausalLM, PreTrainedTokenizerFast
from peft import LoraConfig, get_peft_model
import torch.nn as nn

# -------------------------
# CONFIG - 请按需修改
# -------------------------
CONFIG = {
    "base_model_path": "/root/autodl-tmp/Pro/qwen2.5-sft-full",   # 模型一：已微调的诗歌模型（基座）
    "ulysses_txt": "/root/autodl-tmp/Pro/Ulysses尤利西斯.txt",    # 尤利西斯原文 txt（本地）
    "output_dir": "/root/autodl-tmp/Pro/qwen2.5-mindflow-ulysses",# 输出模型（二）
    "max_len": 1024,      # 单个样本最大 token 长度（可调：1024/2048）
    "stride": 512,        # 滑动窗口重叠
    "batch_size": 1,      # per device batch size（通常 1 或 2）
    "gradient_accumulation_steps": 4,
    "learning_rate": 2e-5,
    "num_train_epochs": 2,
    "fp16": True,
    "lora_r": 4,
    "lora_alpha": 16,
    "lora_dropout": 0.01,
    "target_modules": ["q_proj", "v_proj"],
    "print_every_steps": 20,
    "seed": 42,
}

os.makedirs(CONFIG["output_dir"], exist_ok=True)
device = "cuda" if torch.cuda.is_available() else "cpu"
torch.manual_seed(CONFIG["seed"])
random.seed(CONFIG["seed"])

# -------------------------
# 1. 加载 tokenizer（Qwen2 正确范式）
# -------------------------
print("Loading tokenizer from", CONFIG["base_model_path"])

tokenizer_path = os.path.join(CONFIG["base_model_path"], "tokenizer.json")
if os.path.exists(tokenizer_path):
    tokenizer = PreTrainedTokenizerFast(tokenizer_file=tokenizer_path)
else:
    tokenizer = AutoTokenizer.from_pretrained(CONFIG["base_model_path"], trust_remote_code=True)

# ======【Qwen2 推荐标准设置】======
# 1) 明确设置 eos / bos（大多数 qwen2 tokenizer.json 自带，但确保一致）
tokenizer.eos_token = "<|endoftext|>"
tokenizer.bos_token = "<|startoftext|>"

# 2) 添加 im_start / im_end
# 注意：这一步会重建 vocab → pad_token 会丢失
tokenizer.add_special_tokens({
    "additional_special_tokens": ["<|im_start|>", "<|im_end|>"]
})

# 3) ★ 关键：add_special_tokens 后必须重新设置 pad_token
tokenizer.pad_token = tokenizer.eos_token

print("Tokenizer loaded.")
print("  eos_token:", tokenizer.eos_token)
print("  bos_token:", tokenizer.bos_token)
print("  pad_token:", tokenizer.pad_token)
print("  pad_token_id:", tokenizer.pad_token_id)
print("  vocab_size:", len(tokenizer))
# =================================

# -------------------------
# 2. 从 Ulysses txt 生成 token chunks（滑动窗口）
# -------------------------
def load_and_tokenize_txt(txt_path, tokenizer, max_len, stride):
    print("Reading Ulysses txt:", txt_path)
    with open(txt_path, "r", encoding="utf-8") as f:
        raw = f.read()

    # 简单按段落拆分（保留语义完整性），段落间保留换行
    paragraphs = [p.strip() for p in raw.split("\n\n") if p.strip()]
    print(f"Found {len(paragraphs)} paragraphs; will pack paragraphs into chunks <= {max_len} tokens.")

    all_chunks = []
    cur_ids = []
    cur_chars = []

    for para in paragraphs:
        para_ids = tokenizer(para, add_special_tokens=False)["input_ids"]
        if len(para_ids) > max_len:
            # 段落本身超长：按 token 窗口切分
            start = 0
            while start < len(para_ids):
                end = min(start + max_len, len(para_ids))
                chunk_ids = para_ids[start:end]
                chunk_text = tokenizer.decode(chunk_ids, skip_special_tokens=True)
                all_chunks.append(chunk_text)
                if end == len(para_ids):
                    break
                start += max_len - stride
            continue

        # 若加入当前 chunk 后不超长则合并
        if len(cur_ids) + len(para_ids) <= max_len:
            cur_ids.extend(para_ids)
            cur_chars.append(para)
        else:
            # flush current
            if cur_ids:
                all_chunks.append(tokenizer.decode(cur_ids, skip_special_tokens=True))
            # start new
            cur_ids = para_ids.copy()
            cur_chars = [para]

    # flush tail
    if cur_ids:
        all_chunks.append(tokenizer.decode(cur_ids, skip_special_tokens=True))

    # 进一步应用滑动窗口以增加多样性（可选）
    # 这里我们再把每 chunk 做 stride 分割，避免遗漏跨段上下文
    final_chunks = []
    for chunk in all_chunks:
        ids = tokenizer(chunk, add_special_tokens=False)["input_ids"]
        start = 0
        while start < len(ids):
            end = min(start + max_len, len(ids))
            part = tokenizer.decode(ids[start:end], skip_special_tokens=True)
            final_chunks.append(part)
            if end == len(ids):
                break
            start += max_len - stride

    print(f"Total token chunks prepared: {len(final_chunks)}")
    return final_chunks

chunks = load_and_tokenize_txt(CONFIG["ulysses_txt"], tokenizer, CONFIG["max_len"], CONFIG["stride"])

# -------------------------
# 3. 构造 Dataset（对话格式 + prefix-only mask）
# -------------------------
class UlyssesDataset(Dataset):
    def __init__(self, chunks, tokenizer, max_len):
        self.examples = []
        self.tokenizer = tokenizer
        self.max_len = max_len

        for idx, chunk_text in enumerate(chunks):
            # user instruction: 要求生成意识流风格的段落（不抄袭原文）
            user_instr = (
                "Write an original paragraph in James Joyce's stream-of-consciousness style "
                "that captures similar imagery and tone as the following excerpt. "
                "Do not copy exact phrases; be original.\n\n"
                f"Excerpt:\n{chunk_text}"
            )

            # We will teach the model to generate text in this style by using the chunk_text
            # as the assistant target. (You can replace this by paraphrases if desired.)
            assistant_target = chunk_text

            full = f"<|im_start|>user\n{user_instr}<|im_end|>\n<|im_start|>assistant\n{assistant_target}<|im_end|>"
            enc = tokenizer(full, max_length=self.max_len, truncation=True, padding="max_length", return_tensors="pt")

            input_ids = enc["input_ids"].squeeze(0)
            attention_mask = enc["attention_mask"].squeeze(0)
            labels = input_ids.clone()

            # find assistant start token index to mask prefix
            # We search for the tokenizer encoding of "<|im_start|>assistant\n"
            # Simpler: find the first occurrence of "<|im_start|>" after the user block.
            special_id = tokenizer.convert_tokens_to_ids("<|im_start|>")
            # locate positions
            ids_list = input_ids.tolist()
            assistant_pos = 0
            try:
                # find second occurrence of im_start (first is user)
                first = ids_list.index(special_id)
                second = ids_list.index(special_id, first + 1)
                assistant_pos = second
            except ValueError:
                # fallback: try to find the textual marker
                txt = tokenizer.decode(input_ids, skip_special_tokens=False)
                marker = "<|im_start|>assistant\n"
                if marker in txt:
                    assistant_pos = tokenizer(marker, add_special_tokens=False)["input_ids"][0]
                    # not reliable; fallback to 0
                else:
                    assistant_pos = 0

            # mask prefix (user + instruction) so loss is only computed on assistant part
            labels[:assistant_pos] = -100

            self.examples.append({
                "input_ids": input_ids,
                "attention_mask": attention_mask,
                "labels": labels
            })

        print(f"Dataset built: {len(self.examples)} examples")

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        return self.examples[idx]

dataset = UlyssesDataset(chunks, tokenizer, CONFIG["max_len"])
dataloader = DataLoader(dataset, batch_size=CONFIG["batch_size"], shuffle=True, num_workers=0, drop_last=True)

# -------------------------
# 4. 加载基座模型并装上 LoRA
# -------------------------
print("Loading base model from:", CONFIG["base_model_path"])
model = AutoModelForCausalLM.from_pretrained(
    CONFIG["base_model_path"],
    torch_dtype=torch.float16 if CONFIG["fp16"] else torch.float32,
    device_map="auto",
    low_cpu_mem_usage=True
)

lora_cfg = LoraConfig(
    r=CONFIG["lora_r"],
    lora_alpha=CONFIG["lora_alpha"],
    lora_dropout=CONFIG["lora_dropout"],
    target_modules=CONFIG["target_modules"],
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, lora_cfg)
model.print_trainable_parameters()
model.train()

# device for inputs: model may be sharded; use next(model.parameters()).device
model_device = next(model.parameters()).device
print("Model device:", model_device)

# -------------------------
# 5. optimizer & loss
# -------------------------
optimizer = torch.optim.AdamW(model.parameters(), lr=CONFIG["learning_rate"])
loss_fn = nn.CrossEntropyLoss(ignore_index=-100)

# -------------------------
# 6. 训练循环
# -------------------------
def format_time(seconds):
    m, s = divmod(int(seconds), 60)
    h, m = divmod(m, 60)
    return f"{h:02d}:{m:02d}:{s:02d}"

total_steps = len(dataloader) * CONFIG["num_train_epochs"] // CONFIG["gradient_accumulation_steps"]
print("Training config -- epochs:", CONFIG["num_train_epochs"], "batches_per_epoch:", len(dataloader), "total_steps(approx):", total_steps)

global_step = 0
for epoch in range(CONFIG["num_train_epochs"]):
    epoch_start = time.time()
    running_loss = 0.0
    optimizer.zero_grad()

    for step, batch in enumerate(dataloader):
        global_step += 1

        input_ids = batch["input_ids"].to(model_device)
        attention_mask = batch["attention_mask"].to(model_device)
        labels = batch["labels"].to(model_device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss / CONFIG["gradient_accumulation_steps"]
        loss.backward()

        if global_step % CONFIG["gradient_accumulation_steps"] == 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.5)
            optimizer.step()
            optimizer.zero_grad()

        running_loss += loss.item() * CONFIG["gradient_accumulation_steps"]

        if global_step % CONFIG["print_every_steps"] == 0:
            avg_loss = running_loss / (global_step if global_step>0 else 1)
            print(f"[Epoch {epoch+1}] Step {global_step} | loss: {avg_loss:.4f} | step_time: {format_time(time.time()-epoch_start)}")

    epoch_time = format_time(time.time() - epoch_start)
    print(f"Epoch {epoch+1} finished in {epoch_time}")

# -------------------------
# 7. 保存微调后模型（包含 LoRA）
# -------------------------
print("Saving finetuned model to:", CONFIG["output_dir"])
model.save_pretrained(CONFIG["output_dir"])
tokenizer.save_pretrained(CONFIG["output_dir"])
print("Done.")


Loading tokenizer from /root/autodl-tmp/Pro/qwen2.5-sft-full
Tokenizer loaded.
  eos_token: <|endoftext|>
  bos_token: <|startoftext|>
  pad_token: <|endoftext|>
  pad_token_id: 151643
  vocab_size: 151665
Reading Ulysses txt: /root/autodl-tmp/Pro/Ulysses尤利西斯.txt
Found 7169 paragraphs; will pack paragraphs into chunks <= 1024 tokens.
Total token chunks prepared: 459


`torch_dtype` is deprecated! Use `dtype` instead!


Dataset built: 459 examples
Loading base model from: /root/autodl-tmp/Pro/qwen2.5-sft-full


Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.12it/s]
/root/miniconda3/envs/qwen_text/lib/python3.11/site-packages/peft/mapping_func.py:72: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/root/miniconda3/envs/qwen_text/lib/python3.11/site-packages/peft/tuners/tuners_utils.py:282: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


trainable params: 1,261,568 || all params: 7,616,878,080 || trainable%: 0.0166
Model device: cuda:0
Training config -- epochs: 2 batches_per_epoch: 459 total_steps(approx): 229
[Epoch 1] Step 20 | loss: 2.4292 | step_time: 00:00:04
[Epoch 1] Step 40 | loss: 2.8493 | step_time: 00:00:07
[Epoch 1] Step 60 | loss: 3.1239 | step_time: 00:00:11
[Epoch 1] Step 80 | loss: 2.8827 | step_time: 00:00:15
[Epoch 1] Step 100 | loss: 2.8391 | step_time: 00:00:19
[Epoch 1] Step 120 | loss: 2.8489 | step_time: 00:00:22
[Epoch 1] Step 140 | loss: 2.8245 | step_time: 00:00:26
[Epoch 1] Step 160 | loss: 2.8936 | step_time: 00:00:30
[Epoch 1] Step 180 | loss: 2.7989 | step_time: 00:00:34
[Epoch 1] Step 200 | loss: 2.7810 | step_time: 00:00:37
[Epoch 1] Step 220 | loss: 2.7274 | step_time: 00:00:41
[Epoch 1] Step 240 | loss: 2.6937 | step_time: 00:00:45
[Epoch 1] Step 260 | loss: 2.6742 | step_time: 00:00:48
[Epoch 1] Step 280 | loss: 2.6603 | step_time: 00:00:52
[Epoch 1] Step 300 | loss: 2.6799 | step_ti

In [ ]:
#加要求的意识流风格微调优化方向（LoRA 升级、损失函数扩展、训练参数调整）进行适配
# finetune_mindflow_ulysses.py
import os
import json
import time
import torch
import random
import re
import nltk
from nltk.corpus import wordnet
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForCausalLM, PreTrainedTokenizerFast
from peft import LoraConfig, get_peft_model
import torch.nn as nn
import torch.nn.functional as F

# -------------------------
# CONFIG - 意识流风格微调优化配置（修复LoRA目标模块）
# -------------------------
CONFIG = {
    "base_model_path": "/root/autodl-tmp/Pro/qwen2.5-sft-full",   # 模型一：已微调的诗歌模型（基座）
    "ulysses_txt": "/root/autodl-tmp/Pro/Ulysses尤利西斯.txt",    # 尤利西斯原文 txt（本地）
    "output_dir": "/root/autodl-tmp/Pro/qwen2.5-mindflow-ulysses",# 输出模型（二）
    "max_len": 1024,      # 单个样本最大 token 长度（可调：1024/2048）
    "stride": 512,        # 滑动窗口重叠
    "batch_size": 1,      # per device batch size（通常 1 或 2）
    "gradient_accumulation_steps": 4,
    "learning_rate": 1e-5,  # 优化：降至1e-5（慢学习，精准拟合风格）
    "num_train_epochs": 5,   # 优化：增至5轮
    "fp16": True,
    "lora_r": 16,            # 优化：提升至16（增强风格拟合）
    "lora_alpha": 16,
    "lora_dropout": 0.01,
    # 修复：Qwen2.5的MLP层具体Linear子层名称（替换原mlp）
    "target_modules": ["q_proj", "v_proj", "k_proj", "o_proj", 
                       "gate_proj", "up_proj", "down_proj"],  
    "print_every_steps": 20,
    "seed": 42,
    # 新增：多样性损失&句式风格损失参数
    "diversity_alpha": 0.1,  # 多样性损失α提升至0.1
    "style_beta": 0.05,      # 句式风格损失β系数
    "eps": 1e-6,             # 数值稳定参数
    "max_norm": 1.5,         # 梯度裁剪阈值
    # 同义词替换参数（保留NLTK wordnet能力）
    "augment_prob": 0.5      # 50%概率进行同义词替换增强
}

os.makedirs(CONFIG["output_dir"], exist_ok=True)
device = "cuda" if torch.cuda.is_available() else "cpu"
torch.manual_seed(CONFIG["seed"])
random.seed(CONFIG["seed"])

# -------------------------
# 仅下载NLTK wordnet（同义词替换用），不下载punkt
# -------------------------
try:
    nltk.data.find('corpora/wordnet')
except LookupError:
    nltk.download('wordnet')
try:
    nltk.data.find('taggers/averaged_perceptron_tagger')
except LookupError:
    nltk.download('averaged_perceptron_tagger')

# -------------------------
# 保留：英文文本同义词替换增强（基于NLTK wordnet）
# -------------------------
def augment_english_text(text):
    """基于NLTK wordnet的同义词替换，保留语义同时提升多样性"""
    if not text or len(text.split()) < 3:
        return text
    
    try:
        # 词性标注，仅替换名词/动词/形容词
        pos_tags = nltk.pos_tag(text.split())
        augmented_tokens = []
        for token, pos in pos_tags:
            if pos.startswith('N') or pos.startswith('V') or pos.startswith('J'):
                synonyms = wordnet.synsets(token)
                if synonyms and len(synonyms) > 0:
                    synonym_lemma = synonyms[0].lemmas()[0].name()
                    # 严格校验替换词
                    if (synonym_lemma != token and 
                        len(synonym_lemma) <= len(token)+2 and
                        not synonym_lemma.isdigit()):
                        augmented_tokens.append(synonym_lemma.replace('_', ' '))
                    else:
                        augmented_tokens.append(token)
                else:
                    augmented_tokens.append(token)
            else:
                augmented_tokens.append(token)
        return ' '.join(augmented_tokens)
    except:
        return text  # 增强失败时返回原文本

# -------------------------
# 1. 加载 tokenizer（Qwen2 正确范式）
# -------------------------
print("Loading tokenizer from", CONFIG["base_model_path"])

tokenizer_path = os.path.join(CONFIG["base_model_path"], "tokenizer.json")
if os.path.exists(tokenizer_path):
    tokenizer = PreTrainedTokenizerFast(tokenizer_file=tokenizer_path)
else:
    tokenizer = AutoTokenizer.from_pretrained(CONFIG["base_model_path"], trust_remote_code=True)

# ======【Qwen2 推荐标准设置】======
# 1) 明确设置 eos / bos（大多数 qwen2 tokenizer.json 自带，但确保一致）
tokenizer.eos_token = "<|endoftext|>"
tokenizer.bos_token = "<|startoftext|>"

# 2) 添加 im_start / im_end
# 注意：这一步会重建 vocab → pad_token 会丢失
tokenizer.add_special_tokens({
    "additional_special_tokens": ["<|im_start|>", "<|im_end|>"]
})

# 3) ★ 关键：add_special_tokens 后必须重新设置 pad_token
tokenizer.pad_token = tokenizer.eos_token

print("Tokenizer loaded.")
print("  eos_token:", tokenizer.eos_token)
print("  bos_token:", tokenizer.bos_token)
print("  pad_token:", tokenizer.pad_token)
print("  pad_token_id:", tokenizer.pad_token_id)
print("  vocab_size:", len(tokenizer))
# =================================

# -------------------------
# 2. 从 Ulysses txt 生成 token chunks（滑动窗口 + 文本增强）
# -------------------------
def load_and_tokenize_txt(txt_path, tokenizer, max_len, stride):
    print("Reading Ulysses txt:", txt_path)
    with open(txt_path, "r", encoding="utf-8") as f:
        raw = f.read()

    # 简单按段落拆分（保留语义完整性），段落间保留换行
    paragraphs = [p.strip() for p in raw.split("\n\n") if p.strip()]
    print(f"Found {len(paragraphs)} paragraphs; will pack paragraphs into chunks <= {max_len} tokens.")

    all_chunks = []
    cur_ids = []
    cur_chars = []

    for para in paragraphs:
        # 对段落进行同义词替换增强（保留意识流风格）
        if random.random() < CONFIG["augment_prob"]:
            para = augment_english_text(para)
        
        para_ids = tokenizer(para, add_special_tokens=False)["input_ids"]
        if len(para_ids) > max_len:
            # 段落本身超长：按 token 窗口切分
            start = 0
            while start < len(para_ids):
                end = min(start + max_len, len(para_ids))
                chunk_ids = para_ids[start:end]
                chunk_text = tokenizer.decode(chunk_ids, skip_special_tokens=True)
                all_chunks.append(chunk_text)
                if end == len(para_ids):
                    break
                start += max_len - stride
            continue

        # 若加入当前 chunk 后不超长则合并
        if len(cur_ids) + len(para_ids) <= max_len:
            cur_ids.extend(para_ids)
            cur_chars.append(para)
        else:
            # flush current
            if cur_ids:
                all_chunks.append(tokenizer.decode(cur_ids, skip_special_tokens=True))
            # start new
            cur_ids = para_ids.copy()
            cur_chars = [para]

    # flush tail
    if cur_ids:
        all_chunks.append(tokenizer.decode(cur_ids, skip_special_tokens=True))

    # 进一步应用滑动窗口以增加多样性（可选）
    final_chunks = []
    for chunk in all_chunks:
        ids = tokenizer(chunk, add_special_tokens=False)["input_ids"]
        start = 0
        while start < len(ids):
            end = min(start + max_len, len(ids))
            part = tokenizer.decode(ids[start:end], skip_special_tokens=True)
            final_chunks.append(part)
            if end == len(ids):
                break
            start += max_len - stride

    print(f"Total token chunks prepared: {len(final_chunks)}")
    return final_chunks

chunks = load_and_tokenize_txt(CONFIG["ulysses_txt"], tokenizer, CONFIG["max_len"], CONFIG["stride"])

# -------------------------
# 调整：句子长度分析工具（无需punkt，基于标点/空格分词）
# -------------------------
def analyze_sentence_length(text):
    """
    分析文本中的短句占比（无需punkt）
    短句定义：≤5个单词的文本片段（按 . ! ? ; , 分割）
    """
    # 清理特殊符号，保留核心文本
    clean_text = re.sub(r'<\|.*?\|>', '', text)
    clean_text = re.sub(r'\s+', ' ', clean_text).strip()
    
    # 按标点分割文本片段（模拟分句，无需punkt）
    fragments = re.split(r'[.!?;,]', clean_text)
    fragments = [f.strip() for f in fragments if f.strip()]
    total_fragments = len(fragments)
    
    if total_fragments == 0:
        return 0.0
    
    # 统计短句数量
    short_frag_count = 0
    for frag in fragments:
        # 分词（仅保留单词）
        words = re.findall(r'\w+', frag)
        if len(words) <= 5:
            short_frag_count += 1
    
    # 返回短句占比
    return short_frag_count / total_fragments

# -------------------------
# 新增：损失函数扩展（多样性损失 + 句式风格损失）
# -------------------------
def diversity_loss(logits, alpha=CONFIG["diversity_alpha"]):
    """多样性损失：惩罚低熵预测，增强生成多样性"""
    # 转换为fp32计算，避免数值溢出
    logits = logits.float()
    
    # 计算softmax概率分布
    probs = F.softmax(logits, dim=-1)
    # 限制概率范围，防止log(0)
    probs = torch.clamp(probs, min=CONFIG["eps"], max=1.0 - CONFIG["eps"])
    
    # 计算熵值
    entropy = -torch.sum(probs * torch.log(probs), dim=-1)
    # 最大熵（词汇表大小的对数）
    vocab_size = probs.size(-1)
    max_entropy = torch.log(torch.tensor(vocab_size, dtype=torch.float32, device=logits.device))
    # 归一化熵值
    entropy_norm = entropy / (max_entropy + CONFIG["eps"])
    entropy_norm = torch.clamp(entropy_norm, min=0.0, max=1.0)
    
    # 惩罚低熵值（保守预测）
    penalty = alpha * (1 - entropy_norm)
    penalty = torch.nan_to_num(penalty, nan=0.0, posinf=alpha, neginf=0.0)
    
    return torch.mean(penalty)

def style_loss(generated_texts, beta=CONFIG["style_beta"]):
    """句式风格损失：惩罚长句占比过高，强化碎片化"""
    total_style_penalty = 0.0
    valid_text_count = 0
    
    for text in generated_texts:
        short_frag_ratio = analyze_sentence_length(text)
        # 惩罚项：1 - 短句占比（短句越少，惩罚越高）
        penalty = 1 - short_frag_ratio
        total_style_penalty += penalty
        valid_text_count += 1
    
    if valid_text_count == 0:
        return torch.tensor(0.0, device=device)
    
    # 计算平均惩罚并乘以beta系数
    avg_penalty = total_style_penalty / valid_text_count
    return beta * torch.tensor(avg_penalty, device=device, dtype=torch.float32)

# -------------------------
# 3. 构造 Dataset（对话格式 + prefix-only mask）
# -------------------------
class UlyssesDataset(Dataset):
    def __init__(self, chunks, tokenizer, max_len):
        self.examples = []
        self.tokenizer = tokenizer
        self.max_len = max_len

        for idx, chunk_text in enumerate(chunks):
            # user instruction: 要求生成意识流风格的段落（不抄袭原文）
            user_instr = (
                "Write an original paragraph in James Joyce's stream-of-consciousness style "
                "that captures similar imagery and tone as the following excerpt. "
                "Do not copy exact phrases; be original.\n\n"
                f"Excerpt:\n{chunk_text}"
            )

            # 目标文本（可替换为增强后的文本）
            assistant_target = chunk_text

            full = f"<|im_start|>user\n{user_instr}<|im_end|>\n<|im_start|>assistant\n{assistant_target}<|im_end|>"
            enc = tokenizer(full, max_length=self.max_len, truncation=True, padding="max_length", return_tensors="pt")

            input_ids = enc["input_ids"].squeeze(0)
            attention_mask = enc["attention_mask"].squeeze(0)
            labels = input_ids.clone()

            # 定位assistant部分起始位置，mask掉user部分的loss
            special_id = tokenizer.convert_tokens_to_ids("<|im_start|>")
            ids_list = input_ids.tolist()
            assistant_pos = 0
            try:
                # 找到第二个<|im_start|>（第一个是user）
                first = ids_list.index(special_id)
                second = ids_list.index(special_id, first + 1)
                assistant_pos = second
            except ValueError:
                assistant_pos = 0

            # mask prefix（user + instruction）so loss is only computed on assistant part
            labels[:assistant_pos] = -100

            self.examples.append({
                "input_ids": input_ids,
                "attention_mask": attention_mask,
                "labels": labels,
                "assistant_text": assistant_target  # 保存目标文本用于风格损失计算
            })

        print(f"Dataset built: {len(self.examples)} examples")

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        return self.examples[idx]

dataset = UlyssesDataset(chunks, tokenizer, CONFIG["max_len"])
dataloader = DataLoader(dataset, batch_size=CONFIG["batch_size"], shuffle=True, num_workers=0, drop_last=True)

# -------------------------
# 4. 加载基座模型并装上 LoRA（修复配置）
# -------------------------
print("Loading base model from:", CONFIG["base_model_path"])
model = AutoModelForCausalLM.from_pretrained(
    CONFIG["base_model_path"],
    torch_dtype=torch.float16 if CONFIG["fp16"] else torch.float32,
    device_map="auto",
    low_cpu_mem_usage=True,
    trust_remote_code=True  # 关键：Qwen2需要trust_remote_code
)

# 修复：LoRA配置（目标模块为Qwen2.5的具体Linear层）
lora_cfg = LoraConfig(
    r=CONFIG["lora_r"],
    lora_alpha=CONFIG["lora_alpha"],
    lora_dropout=CONFIG["lora_dropout"],
    target_modules=CONFIG["target_modules"],  # q_proj/v_proj/k_proj/o_proj + MLP的Linear层
    task_type="CAUSAL_LM",
    bias="none",  # 避免bias训练导致的数值问题
    modules_to_save=[],  # 不额外保存模块
)
model = get_peft_model(model, lora_cfg)
model.print_trainable_parameters()  # 打印可训练参数占比
model.train()

# 获取模型设备
model_device = next(model.parameters()).device
print("Model device:", model_device)

# -------------------------
# 5. optimizer & loss
# -------------------------
optimizer = torch.optim.AdamW(model.parameters(), lr=CONFIG["learning_rate"])
ce_loss_fn = nn.CrossEntropyLoss(ignore_index=-100)

# -------------------------
# 6. 训练循环（集成新损失函数）
# -------------------------
def format_time(seconds):
    m, s = divmod(int(seconds), 60)
    h, m = divmod(m, 60)
    return f"{h:02d}:{m:02d}:{s:02d}"

total_steps = len(dataloader) * CONFIG["num_train_epochs"] // CONFIG["gradient_accumulation_steps"]
print("Training config -- epochs:", CONFIG["num_train_epochs"], "batches_per_epoch:", len(dataloader), "total_steps(approx):", total_steps)

global_step = 0
for epoch in range(CONFIG["num_train_epochs"]):
    epoch_start = time.time()
    running_loss = 0.0
    running_ce_loss = 0.0
    running_div_loss = 0.0
    running_style_loss = 0.0
    optimizer.zero_grad()

    for step, batch in enumerate(dataloader):
        global_step += 1

        input_ids = batch["input_ids"].to(model_device)
        attention_mask = batch["attention_mask"].to(model_device)
        labels = batch["labels"].to(model_device)
        assistant_texts = batch["assistant_text"]  # 目标文本用于风格损失计算

        # 前向传播
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        logits = outputs.logits
        ce_loss = ce_loss_fn(logits.view(-1, logits.size(-1)), labels.view(-1))
        
        # 计算多样性损失
        div_loss = diversity_loss(logits.view(-1, logits.size(-1)))
        
        # 计算句式风格损失
        style_loss_val = style_loss(assistant_texts)
        
        # 总损失：交叉熵 + 多样性损失 + 风格损失
        total_loss = (ce_loss + div_loss + style_loss_val) / CONFIG["gradient_accumulation_steps"]
        total_loss.backward()

        # 梯度裁剪 + 优化器更新
        if global_step % CONFIG["gradient_accumulation_steps"] == 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), CONFIG["max_norm"])
            optimizer.step()
            optimizer.zero_grad()

        # 累计损失
        running_loss += total_loss.item() * CONFIG["gradient_accumulation_steps"]
        running_ce_loss += ce_loss.item()
        running_div_loss += div_loss.item()
        running_style_loss += style_loss_val.item()

        # 打印训练日志
        if global_step % CONFIG["print_every_steps"] == 0:
            avg_loss = running_loss / (global_step if global_step>0 else 1)
            avg_ce = running_ce_loss / (global_step if global_step>0 else 1)
            avg_div = running_div_loss / (global_step if global_step>0 else 1)
            avg_style = running_style_loss / (global_step if global_step>0 else 1)
            print(f"[Epoch {epoch+1}] Step {global_step} | total_loss: {avg_loss:.4f} | ce_loss: {avg_ce:.4f} | div_loss: {avg_div:.4f} | style_loss: {avg_style:.4f} | time: {format_time(time.time()-epoch_start)}")

    epoch_time = format_time(time.time() - epoch_start)
    print(f"Epoch {epoch+1} finished in {epoch_time} | avg_epoch_loss: {running_loss/len(dataloader):.4f}")

# -------------------------
# 7. 保存微调后模型（包含 LoRA）
# -------------------------
print("Saving finetuned model to:", CONFIG["output_dir"])
model.save_pretrained(CONFIG["output_dir"])
tokenizer.save_pretrained(CONFIG["output_dir"])
print("Done.")

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

# ====================== 核心配置 ======================
BASE_MODEL_PATH = "/root/autodl-tmp/Pro/qwen2.5-sft-full"
LORA_VER2_PATH = "/root/autodl-tmp/Pro/qwen2.5-mindflow-ulysses-ver2"
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")  # 强制指定cuda:0

# ====================== 加载tokenizer ======================
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_PATH, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

# ====================== 加载模型（强制单设备） ======================
# 加载基座模型（关闭auto设备映射，强制指定DEVICE）
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_PATH,
    torch_dtype=torch.float16,
    device_map={"": DEVICE},  # 强制所有层都在指定设备
    trust_remote_code=True
)

# 加载LoRA并移到指定设备
model = PeftModel.from_pretrained(model, LORA_VER2_PATH)
model = model.to(DEVICE)  # 强制模型整体移到DEVICE
model.eval()

# ====================== 生成函数 ======================
def generate_poem(prompt):
    # 封装prompt
    input_text = f"<|im_start|>user\n{prompt}<|im_end|>\n<|im_start|>assistant\n"
    
    # 编码（强制移到DEVICE）
    inputs = tokenizer(
        input_text,
        return_tensors="pt",
        padding=True,
        truncation=True
    ).to(DEVICE)  # 关键：输入张量移到DEVICE
    
    # 生成
    with torch.no_grad():
        output = model.generate(
            **inputs,  # 直接传字典，包含input_ids和attention_mask
            max_new_tokens=200,
            temperature=0.8,
            top_p=0.9,
            repetition_penalty=1.0,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id,
            do_sample=True,
            device=DEVICE  # 显式指定生成设备
        )
    
    # 解析结果
    poem = tokenizer.decode(output[0], skip_special_tokens=True)
    poem = poem.split("<|im_start|>assistant\n")[-1].strip()
    return poem

# ====================== 测试 ======================
if __name__ == "__main__":
    result = generate_poem("写一首诗")
    print("=== 生成的诗歌 ===")
    print(result)

In [2]:
# inference_mindflow.py
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

# -------------------------
# 1. 配置路径（新增原始诗歌LoRA路径）
# -------------------------
BASE_MODEL = "/root/autodl-tmp/Pro/qwen2.5-sft-full"               # 基础诗歌微调基座（模型一）
LORA_POETRY = ""  # 【必填】原始诗歌LoRA适配器路径（若模型一是全量微调则留空）
LORA_MINDFLOW = "/root/autodl-tmp/Pro/qwen2.5-mindflow-ulysses"  # 意识流增强LoRA（模型二）
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

GEN_CFG = {
    "max_new_tokens": 500,
    "temperature": 0.85,
    "top_k": 40,
    "top_p": 0.92,
    "repetition_penalty": 1.02,
    "do_sample": True,  # 确保采样生成，增强多样性
}

# -------------------------
# 2. 加载通用tokenizer
# -------------------------
print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(
    BASE_MODEL,
    trust_remote_code=True
)

# 必须设置 pad_token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# -------------------------
# 3. 模型加载工具函数（复用基座，切换LoRA）
# -------------------------
def load_model_with_lora(base_model_path, lora_path=None):
    """
    加载基座模型 + 指定LoRA适配器
    :param base_model_path: 基座模型路径
    :param lora_path: LoRA适配器路径（None则仅加载基座）
    :return: 加载完成的模型
    """
    # 加载基座模型
    model = AutoModelForCausalLM.from_pretrained(
        base_model_path,
        torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True
    )
    
    # 加载LoRA（若指定）
    if lora_path and lora_path.strip() != "":
        model = PeftModel.from_pretrained(model, lora_path)
    
    model.eval()
    return model

# -------------------------
# 4. 统一推理函数
# -------------------------
def generate_poem(model, tokenizer, prompt: str, gen_cfg: dict):
    """
    统一生成函数，适配任意模型
    """
    # 保持和训练一致的对话格式
    formatted = (
        f"<|im_start|>user\n{prompt}<|im_end|>\n"
        f"<|im_start|>assistant\n"
    )

    input_ids = tokenizer(
        formatted,
        return_tensors="pt"
    ).input_ids.to(DEVICE)

    with torch.no_grad():
        output = model.generate(
            input_ids=input_ids,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id,
            **gen_cfg
        )

    # 清理生成结果
    text = tokenizer.decode(output[0], skip_special_tokens=False)
    if "<|im_start|>assistant\n" in text:
        text = text.split("<|im_start|>assistant\n")[1]
    text = text.replace("<|im_end|>", "").replace("<|endoftext|>", "").strip()
    
    return text

# -------------------------
# 5. 主推理流程（双模型对比）
# -------------------------
if __name__ == "__main__":
    # 测试Prompt
    test_prompt = (
        "write a poem in the style of virginia wolf, with keyword: willow, wind, fluffy, charmed."
    )
    
    print("="*80)
    print(f"Test Prompt:\n{test_prompt}")
    print("="*80)

    # 加载模型1：原始诗歌微调模型
    print("\n[Step 1/3] Loading Original Poetry Model...")
    model_poetry = load_model_with_lora(BASE_MODEL, LORA_POETRY)
    
    # 加载模型2：意识流增强微调模型
    print("\n[Step 2/3] Loading Mindflow Enhanced Model...")
    model_mindflow = load_model_with_lora(BASE_MODEL, LORA_MINDFLOW)

    # 生成诗歌（双模型）
    print("\n[Step 3/3] Generating Poems...")
    print("\n=== 1. Original Poetry Model Output ===")
    poem_poetry = generate_poem(model_poetry, tokenizer, test_prompt, GEN_CFG)
    print(poem_poetry)

    print("\n=== 2. Mindflow Enhanced Model Output ===")
    poem_mindflow = generate_poem(model_mindflow, tokenizer, test_prompt, GEN_CFG)
    print(poem_mindflow)

    # 释放显存（可选）
    del model_poetry, model_mindflow
    torch.cuda.empty_cache()

Loading tokenizer...
Test Prompt:
write a poem in the style of virginia wolf, with keyword: willow, wind, fluffy, charmed.

[Step 1/3] Loading Original Poetry Model...


Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.25it/s]



[Step 2/3] Loading Mindflow Enhanced Model...


Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.25it/s]
/root/miniconda3/envs/qwen_text/lib/python3.11/site-packages/peft/tuners/tuners_utils.py:282: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(



[Step 3/3] Generating Poems...

=== 1. Original Poetry Model Output ===
All the world was green, and so was she. She was all wind, and wind was all her hair.
The wind blew her dress about like the sails of a ship, and it was all of willow. She went, she went, she was all fluff. She was all fluff, and all of willow. She had no heart, she was all fluff. She was all fluff, she had no heart. She had no heart, and she was all fluff. She was all fluff, she had no heart. She had no heart, and she was all fluff. She was all fluff, she had no heart. She had no heart, she was all fluff. She was all fluff, she had no heart.

=== 2. Mindflow Enhanced Model Output ===
In a garden where the willow weeps and wails,
Under skies that weave their silvery trails,
The wind, a witch, does softly play,
With leaves and boughs that dance away.

A fluffy cloud, like cotton in the breeze,
Drifts over willow, soft as eves,
In a charmed garden, far from madding crowd,
Where shadows lie, and dreams are loud.

The

In [1]:
import torch
import time
import os
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

# -------------------------
# 1. 配置
# -------------------------
BASE_MODEL = "/root/autodl-tmp/Pro/qwen2.5-sft-full"
LORA_MODEL = "/root/autodl-tmp/Pro/qwen2.5-mindflow-ulysses"

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

GEN_CFG = {
    "max_new_tokens": 400,
    "temperature": 0.85,
    "top_k": 40,
    "top_p": 0.92,
    "repetition_penalty": 1.12,
}

# 新增：保存配置
SAVE_FOLDER = "./poetry_results"  # 保存目录
os.makedirs(SAVE_FOLDER, exist_ok=True)  # 自动创建目录（不存在时）

# -------------------------
# 2. 加载 tokenizer
# -------------------------
print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(
    BASE_MODEL,
    trust_remote_code=True
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# -------------------------
# 3. 加载模型 + LoRA
# -------------------------
print("Loading base model...")
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.float16,
    device_map="auto"
)

print("Loading LoRA adapter...")
model = PeftModel.from_pretrained(model, LORA_MODEL)
model.eval()

# -------------------------
# 4. 核心：带“记忆”的生成
# -------------------------
conversation_memory = ""   # ⭐ 存上一轮生成结果


def generate_with_memory(user_prompt: str):
    global conversation_memory

    if conversation_memory:
        # 在已有诗歌基础上 refinement
        formatted = (
            "<|im_start|>user\n"
            "Here is the previous poem:\n"
            f"{conversation_memory}\n\n"
            "Please revise or continue it according to the following instruction:\n"
            f"{user_prompt}"
            "<|im_end|>\n"
            "<|im_start|>assistant\n"
        )
    else:
        # 第一轮
        formatted = (
            f"<|im_start|>user\n{user_prompt}<|im_end|>\n"
            f"<|im_start|>assistant\n"
        )

    input_ids = tokenizer(formatted, return_tensors="pt").input_ids.to(DEVICE)

    with torch.no_grad():
        output = model.generate(
            input_ids=input_ids,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id,
            **GEN_CFG
        )

    text = tokenizer.decode(output[0], skip_special_tokens=False)

    # 清理输出
    if "<|im_start|>assistant\n" in text:
        text = text.split("<|im_start|>assistant\n")[1]
    text = text.replace("<|im_end|>", "").replace("<|endoftext|>", "").strip()

    # ⭐ 更新记忆
    conversation_memory = text

    return text


# -------------------------
# 5. 交互式输入
# -------------------------
if __name__ == "__main__":
    print("\n🧠 Mindflow Poetry Generator")
    print("输入 prompt 回车生成")
    print("输入 reset 清空记忆")
    print("输入 exit 退出并保存终版诗歌\n")

    while True:
        user_input = input(">>> ").strip()

        if user_input.lower() == "exit":
            # 保存终版诗歌到txt文件
            if conversation_memory:
                # 生成带时间戳的文件名，避免覆盖
                timestamp = time.strftime("%Y%m%d_%H%M%S", time.localtime())
                file_path = os.path.join(SAVE_FOLDER, f"final_poem_{timestamp}.txt")
                
                # 写入文件（UTF-8编码防止中文乱码）
                with open(file_path, "w", encoding="utf-8") as f:
                    f.write("=== Mindflow Poetry Final Version ===\n")
                    f.write(f"Generated Time: {time.ctime()}\n")
                    f.write("=" * 40 + "\n\n")
                    f.write(conversation_memory)
                
                print(f"\n✅ 终版诗歌已保存至：{file_path}")
            else:
                print("\n⚠️  暂无生成的诗歌内容，未保存文件")
            
            print("Bye.")
            break

        if user_input.lower() == "reset":
            conversation_memory = ""
            print("🔄 Memory cleared.\n")
            continue

        output = generate_with_memory(user_input)

        print("\n=== Generated ===\n")
        print(output)
        print("\n=================\n")

/root/miniconda3/envs/qwen_text/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading tokenizer...


`torch_dtype` is deprecated! Use `dtype` instead!


Loading base model...


Loading checkpoint shards: 100%|██████████| 4/4 [00:06<00:00,  1.55s/it]


...

>>>  write a poem in the style of james joyce with theme: nostalgic, somber


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



=== Generated ===

In bygone days when life was fair,
I wandered through a land so rare,
Where every stone and every tree,
Was tinged with memories to be.

The sun that shone upon my face,
No more shall gladden me in place.
For time doth run with unrelenting pace,
And all things fade into the space.

Alas, those streets I used to roam,
Now hold no echoes from the home.
The laughter's gone, the song is dead,
A ghostly presence now it's led.

My thoughts they wander far away,
To places where love did play.
Those golden hours that seemed to stretch,
Have slipped away like water in a ditch.

Oh, Dublin town, thou city fair!
How bitter tastes thy memory there.
Once vibrant, filled with hope and dreams,
Now shadows loom o'er all your streams.

The fields we trod, the paths we trode,
Are lost forever from the sod.
Our joys, our sorrows, what remained?
But not within these walls again.

I search for peace amidst this strife,
Yet find but ruins, broken life.
Nostalgic heart that longs to see,


>>>  exit



✅ 终版诗歌已保存至：./poetry_results/final_poem_20251217_192004.txt
Bye.


In [5]:
import torch
import time
import os
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

# -------------------------
# 1. 配置
# -------------------------
# 模型路径配置
MODEL_PATHS = {
    "base": "/root/autodl-tmp/qwen2.5-7b/qwen2.5-7b",
    "sft": "/root/autodl-tmp/Pro/qwen2.5-sft-full",
    "mindflow": {
        "base": "/root/autodl-tmp/Pro/qwen2.5-sft-full",
        "lora": "/root/autodl-tmp/Pro/qwen2.5-mindflow-ulysses"
    }
}

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

GEN_CFG = {
    "max_new_tokens": 400,
    "temperature": 0.85,
    "top_k": 40,
    "top_p": 0.92,
    "repetition_penalty": 1.12,
}

# 新增：保存配置
SAVE_FOLDER = "./poetry_results"  # 保存目录
os.makedirs(SAVE_FOLDER, exist_ok=True)  # 自动创建目录（不存在时）

# -------------------------
# 2. 全局变量：多模型记忆存储
# -------------------------
# 分别存储三个模型的对话记忆
conversation_memory = {
    "base": "",
    "sft": "",
    "mindflow": ""
}
tokenizer = None  # 全局tokenizer
models = {}  # 存储加载的模型

# -------------------------
# 3. 加载 tokenizer
# -------------------------
print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_PATHS["base"],
    trust_remote_code=True
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# -------------------------
# 4. 加载所有模型
# -------------------------
def load_all_models():
    """加载所有三个模型"""
    # 加载Base模型
    print("\nLoading BASE model...")
    models["base"] = AutoModelForCausalLM.from_pretrained(
        MODEL_PATHS["base"],
        torch_dtype=torch.float16,
        device_map="auto"
    ).eval()

    # 加载SFT模型
    print("Loading SFT model...")
    models["sft"] = AutoModelForCausalLM.from_pretrained(
        MODEL_PATHS["sft"],
        torch_dtype=torch.float16,
        device_map="auto"
    ).eval()

    # 加载Mindflow模型（base + LoRA）
    print("Loading MINDFLOW model (base + LoRA)...")
    mindflow_base = AutoModelForCausalLM.from_pretrained(
        MODEL_PATHS["mindflow"]["base"],
        torch_dtype=torch.float16,
        device_map="auto"
    )
    models["mindflow"] = PeftModel.from_pretrained(
        mindflow_base,
        MODEL_PATHS["mindflow"]["lora"]
    ).eval()

    print("\n✅ All models loaded successfully!")
    return models

# -------------------------
# 5. 核心：带“记忆”的多模型生成
# -------------------------
def generate_with_memory(user_prompt: str, model_type: str):
    """带记忆的生成函数（指定模型）"""
    global conversation_memory, models, tokenizer

    model = models[model_type]
    memory = conversation_memory[model_type]

    # 构建带记忆的prompt
    if memory:
        formatted = (
            "<|im_start|>user\n"
            "Here is the previous poem:\n"
            f"{memory}\n\n"
            "Please revise or continue it according to the following instruction:\n"
            f"{user_prompt}"
            "<|im_end|>\n"
            "<|im_start|>assistant\n"
        )
    else:
        # 第一轮生成
        formatted = (
            f"<|im_start|>user\n{user_prompt}<|im_end|>\n"
            f"<|im_start|>assistant\n"
        )

    # 编码输入
    input_ids = tokenizer(formatted, return_tensors="pt").input_ids.to(DEVICE)

    # 生成输出
    with torch.no_grad():
        output = model.generate(
            input_ids=input_ids,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id,
            **GEN_CFG
        )

    # 解码并清理输出
    text = tokenizer.decode(output[0], skip_special_tokens=False)
    if "<|im_start|>assistant\n" in text:
        text = text.split("<|im_start|>assistant\n")[1]
    text = text.replace("<|im_end|>", "").replace("<|endoftext|>", "").strip()

    # 更新对应模型的记忆
    conversation_memory[model_type] = text

    return text

def generate_all_models(user_prompt: str):
    """同时调用三个模型生成结果"""
    results = {}
    
    print("\n" + "-"*80)
    # Base模型生成
    print("🔄 Generating with BASE model...")
    results["base"] = generate_with_memory(user_prompt, "base")
    
    # SFT模型生成
    print("🔄 Generating with SFT model...")
    results["sft"] = generate_with_memory(user_prompt, "sft")
    
    # Mindflow模型生成
    print("🔄 Generating with MINDFLOW model...")
    results["mindflow"] = generate_with_memory(user_prompt, "mindflow")
    print("-"*80 + "\n")
    
    return results

# -------------------------
# 6. 保存所有模型的终版结果
# -------------------------
def save_all_results():
    """保存三个模型的终版输出"""
    timestamp = time.strftime("%Y%m%d_%H%M%S", time.localtime())
    file_path = os.path.join(SAVE_FOLDER, f"all_models_poem_{timestamp}.txt")
    
    with open(file_path, "w", encoding="utf-8") as f:
        # 写入元信息
        f.write("=== Mindflow Poetry - All Models Final Results ===\n")
        f.write(f"Generated Time: {time.ctime()}\n")
        f.write("=" * 60 + "\n\n")
        
        # 写入Base模型结果
        f.write("📌 BASE MODEL OUTPUT:\n")
        f.write("-" * 40 + "\n")
        f.write(conversation_memory["base"] if conversation_memory["base"] else "No content generated\n")
        f.write("\n\n")
        
        # 写入SFT模型结果
        f.write("📌 SFT MODEL OUTPUT:\n")
        f.write("-" * 40 + "\n")
        f.write(conversation_memory["sft"] if conversation_memory["sft"] else "No content generated\n")
        f.write("\n\n")
        
        # 写入Mindflow模型结果
        f.write("📌 MINDFLOW MODEL OUTPUT:\n")
        f.write("-" * 40 + "\n")
        f.write(conversation_memory["mindflow"] if conversation_memory["mindflow"] else "No content generated\n")
    
    return file_path

# -------------------------
# 7. 交互式输入
# -------------------------
if __name__ == "__main__":
    # 加载所有模型
    models = load_all_models()
    
    # 交互式界面
    print("\n\n🧠 Mindflow Poetry Generator - Multi-Model Mode")
    print("=" * 70)
    print("指令说明：")
    print("  1. 直接输入文本      - 同时调用BASE/SFT/MINDFLOW三个模型生成诗歌")
    print("  2. reset             - 清空所有模型的对话记忆")
    print("  3. exit              - 退出并保存所有模型的终版诗歌")
    print("=" * 70 + "\n")

    while True:
        user_input = input(">>> ").strip()

        if not user_input:
            print("⚠️  请输入有效内容！")
            continue

        # 退出并保存所有结果
        if user_input.lower() == "exit":
            # 检查是否有生成内容
            has_content = any(bool(v) for v in conversation_memory.values())
            if has_content:
                file_path = save_all_results()
                print(f"\n✅ 所有模型的终版诗歌已保存至：{file_path}")
            else:
                print("\n⚠️  暂无生成的诗歌内容，未保存文件")
            
            print("\n👋 Bye!")
            break

        # 清空所有记忆
        elif user_input.lower() == "reset":
            conversation_memory = {
                "base": "",
                "sft": "",
                "mindflow": ""
            }
            print("✅ 已清空所有模型的对话记忆！\n")
            continue

        # 正常生成：同时调用三个模型
        else:
            # 生成所有模型的结果
            results = generate_all_models(user_input)
            
            # 输出结果
            print("📝 BASE MODEL OUTPUT:\n")
            print(results["base"])
            print("\n" + "="*80)
            
            print("📝 SFT MODEL OUTPUT:\n")
            print(results["sft"])
            print("\n" + "="*80)
            
            print("📝 MINDFLOW MODEL OUTPUT:\n")
            print(results["mindflow"])
            print("\n" + "="*80 + "\n")

Loading tokenizer...

Loading BASE model...


Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.32it/s]


Loading SFT model...


Loading checkpoint shards: 100%|██████████| 4/4 [00:02<00:00,  1.59it/s]


Loading MINDFLOW model (base + LoRA)...


...

/root/miniconda3/envs/qwen_text/lib/python3.11/site-packages/peft/tuners/tuners_utils.py:282: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


>>>  Write a poem that flows like a stream of thoughts—start with the image of a rainy window, then let your mind wander to memories of childhood, the smell of fresh bread, and the sound of distant thunder. Keep the connection natural. Limit to 8 lines.



--------------------------------------------------------------------------------
🔄 Generating with BASE model...
🔄 Generating with SFT model...
🔄 Generating with MINDFLOW model...
--------------------------------------------------------------------------------

📝 BASE MODEL OUTPUT:

Rain patters on the glass, a lullaby,
Memories swim in soft afternoons gone by.
Childhood summers, warm as honeyed toast,
Baking daydreams beneath the wide, old oak.

Thunder rumbles far away, a sleepy drum,
Familiar scents of yeast rise up from somewhere.
Mom's voice, gentle and kind, through time it calls,
Echoes of laughter mingling with the rain's small fall.

📝 SFT MODEL OUTPUT:

The rain makes a little drum on the windowpane   
And I recall my sister's voice in the kitchen,
When I was eight years old and she made me a slice   
Of sweetbread, her way. The smell of it came through   
To the front room where I sat at the piano playing Schubert.
A song called "Willkommen" and as I played it the rain   
B

>>>  exit



✅ 所有模型的终版诗歌已保存至：./poetry_results/all_models_poem_20251227_115912.txt

👋 Bye!


In [6]:
import json
import os
from sentence_transformers import SentenceTransformer, util
import torch

# -------------------------
# 配置项（根据你的路径修改）
# -------------------------
# 已保存的三首诗歌文件路径
SAVED_POEMS_PATH = "/root/autodl-tmp/Pro/poetry_results/all_models_poem_20251227_115912.txt"
# 训练集jsonl文件路径
TRAIN_DATA_PATH = "/root/autodl-tmp/Pro/poetry_sft.jsonl"
# 相似度模型路径
SIMILARITY_MODEL_PATH = "/root/autodl-tmp/大模型/all-mpnet-base-v2"
# 设备配置
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# -------------------------
# 步骤1：加载相似度模型
# -------------------------
print("🔧 Loading similarity model (all-mpnet-base-v2)...")
similarity_model = SentenceTransformer(SIMILARITY_MODEL_PATH).to(DEVICE)
similarity_model.eval()

# -------------------------
# 步骤2：读取并解析训练集jsonl（提取assistant的诗歌）
# -------------------------
print(f"\n📥 Loading training data from {TRAIN_DATA_PATH}...")
train_poems = []

if not os.path.exists(TRAIN_DATA_PATH):
    raise FileNotFoundError(f"训练集文件不存在：{TRAIN_DATA_PATH}")

with open(TRAIN_DATA_PATH, "r", encoding="utf-8") as f:
    lines = [line.strip() for line in f if line.strip()]
    for line in lines:
        try:
            data = json.loads(line)
            poem_text = ""
            # 提取messages中assistant的content（核心）
            if "messages" in data and isinstance(data["messages"], list):
                for msg in data["messages"]:
                    if msg.get("role") == "assistant" and "content" in msg:
                        poem_text = msg["content"].strip()
                        break
            if poem_text:
                train_poems.append(poem_text)
        except json.JSONDecodeError:
            continue

if not train_poems:
    raise ValueError("训练集中未提取到有效诗歌！请检查jsonl格式")
print(f"✅ 从训练集提取到 {len(train_poems)} 首诗歌")

# 预计算训练集诗歌的嵌入向量（加速相似度计算）
print("\n⚡ Precomputing embeddings for training poems...")
train_embeddings = similarity_model.encode(
    train_poems,
    convert_to_tensor=True,
    device=DEVICE,
    normalize_embeddings=True
)

# -------------------------
# 步骤3：读取已保存的三首生成诗歌
# -------------------------
print(f"\n📤 Loading saved poems from {SAVED_POEMS_PATH}...")
saved_poems = {
    "base": "",
    "sft": "",
    "mindflow": ""
}

if not os.path.exists(SAVED_POEMS_PATH):
    raise FileNotFoundError(f"保存的诗歌文件不存在：{SAVED_POEMS_PATH}")

# 解析保存的文本文件，提取三个模型的诗歌
current_model = None
with open(SAVED_POEMS_PATH, "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        # 识别模型分区
        if "BASE MODEL OUTPUT:" in line:
            current_model = "base"
            continue
        elif "SFT MODEL OUTPUT:" in line:
            current_model = "sft"
            continue
        elif "MINDFLOW MODEL OUTPUT:" in line:
            current_model = "mindflow"
            continue
        # 跳过分隔符、相似度信息等非诗歌内容
        if line.startswith(("=", "-", "📌", "Similarity Score", "Most Similar Poem", "Poem:")):
            continue
        # 收集诗歌内容（保留换行）
        if current_model and line:
            saved_poems[current_model] += line + "\n"

# 清理诗歌内容（去除首尾多余换行）
for model_type in saved_poems:
    saved_poems[model_type] = saved_poems[model_type].strip()
    if not saved_poems[model_type]:
        print(f"⚠️  {model_type.upper()} 模型无有效诗歌内容")

# -------------------------
# 步骤4：计算每首生成诗歌与训练集的相似度
# -------------------------
print("\n" + "="*100)
print("📊 相似度对比结果（生成诗歌 ↔ 训练集最相似诗歌）")
print("="*100)

# 保存最终报告
similarity_report = []
similarity_report.append("=== 诗歌相似度对比报告 ===\n")
similarity_report.append(f"对比时间：{os.popen('date').read().strip()}\n")
similarity_report.append(f"生成诗歌文件：{SAVED_POEMS_PATH}\n")
similarity_report.append(f"训练集文件：{TRAIN_DATA_PATH}\n")
similarity_report.append("="*60 + "\n\n")

for model_type, generated_poem in saved_poems.items():
    if not generated_poem:
        continue
    
    print(f"\n📝 {model_type.upper()} 模型生成诗歌：")
    print("-"*50)
    print(generated_poem)
    print("-"*50)
    
    # 计算生成诗歌的嵌入向量
    gen_embedding = similarity_model.encode(
        generated_poem,
        convert_to_tensor=True,
        device=DEVICE,
        normalize_embeddings=True
    )
    
    # 计算与训练集所有诗歌的余弦相似度
    cos_scores = util.cos_sim(gen_embedding, train_embeddings)[0]
    max_score_idx = torch.argmax(cos_scores).item()
    max_score = round(cos_scores[max_score_idx].item(), 4)
    most_similar_poem = train_poems[max_score_idx]
    
    # 输出相似度结果
    print(f"\n🎯 与训练集最相似诗歌的相似度：{max_score}")
    print(f"\n🆚 训练集中最相似的诗歌：")
    print("-"*50)
    print(most_similar_poem)
    print("-"*50)
    print("\n" + "="*100)
    
    # 写入报告
    similarity_report.append(f"📌 {model_type.upper()} 模型\n")
    similarity_report.append("-"*40 + "\n")
    similarity_report.append(f"生成诗歌：\n{generated_poem}\n\n")
    similarity_report.append(f"相似度分数：{max_score}\n")
    similarity_report.append(f"最相似训练集诗歌：\n{most_similar_poem}\n")
    similarity_report.append("\n" + "="*60 + "\n\n")

# -------------------------
# 步骤5：保存相似度报告
# -------------------------
report_path = os.path.join(os.path.dirname(SAVED_POEMS_PATH), f"similarity_对比报告_{os.path.basename(SAVED_POEMS_PATH)}")
with open(report_path, "w", encoding="utf-8") as f:
    f.writelines(similarity_report)

print(f"\n✅ 相似度对比报告已保存至：{report_path}")

# 释放显存
if DEVICE == "cuda":
    torch.cuda.empty_cache()

🔧 Loading similarity model (all-mpnet-base-v2)...

📥 Loading training data from /root/autodl-tmp/Pro/poetry_sft.jsonl...
✅ 从训练集提取到 13753 首诗歌

⚡ Precomputing embeddings for training poems...

📤 Loading saved poems from /root/autodl-tmp/Pro/poetry_results/all_models_poem_20251227_115912.txt...

📊 相似度对比结果（生成诗歌 ↔ 训练集最相似诗歌）

📝 BASE 模型生成诗歌：
--------------------------------------------------
Rain patters on the glass, a lullaby,
Memories swim in soft afternoons gone by.
Childhood summers, warm as honeyed toast,
Baking daydreams beneath the wide, old oak.
Thunder rumbles far away, a sleepy drum,
Familiar scents of yeast rise up from somewhere.
Mom's voice, gentle and kind, through time it calls,
Echoes of laughter mingling with the rain's small fall.
--------------------------------------------------

🎯 与训练集最相似诗歌的相似度：0.7455

🆚 训练集中最相似的诗歌：
--------------------------------------------------
The rain is haunted;I had forgotten.My children are two hours abedAnd yet I riseHearing behind the typing 

In [ ]:
import numpy as np
import json
import os
from typing import Dict, Optional, Union
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# -------------------------
# 1. 核心配置（适配本地Qwen2.5-7B + 目标诗歌文件）
# -------------------------
# 本地Qwen2.5-7B路径
LOCAL_LLM_PATH = "/root/autodl-tmp/qwen2.5-7b/qwen2.5-7b"
# 目标诗歌文件路径（含三首诗）
POEMS_FILE_PATH = "/root/autodl-tmp/Pro/poetry_results/all_models_poem_20251213_180850.txt"
# 设备配置
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🔧 使用设备：{DEVICE}")

# 论文PIMF 15维度定义（与Table 1完全一致，修正缩进/语法错误）
PIMF_DIMENSIONS = [
    {"key": "creative_imagination", "name": "Creative Imagination", "symbol": "μ₁", "description": "Measures metaphor novelty (e.g., Éluard’s surreal imagery)."},
    {"key": "unpredictability", "name": "Unpredictability", "symbol": "μ₂", "description": "Captures non-formulaic poetic shifts (e.g., Eliot’s fragmentation)."},
    {"key": "autonomy", "name": "Autonomy", "symbol": "μ₃", "description": "Evaluates internal coherence (e.g., Dickinson’s self-contained worlds)."},
    {"key": "poetic_alchemy", "name": "Poetic Alchemy", "symbol": "μ₄", "description": "Transforms ordinary language into deeper meaning (e.g., Dickinson’s 'Hope is the thing with feathers')."},
    {"key": "day_night_imagery", "name": "Day/Night Imagery", "symbol": "μ₅", "description": "Assesses rational vs. dreamlike poetic elements (e.g., Rimbaud’s Illuminations)."},
    {"key": "participative_evocative", "name": "Participative/Evocative Nature", "symbol": "μ₆", "description": "Measures reader engagement and invitation for interpretation (e.g., Rilke’s Archaic Torso of Apollo)."},
    {"key": "assemblage_juxtaposition", "name": "Assemblage/Juxtaposition", "symbol": "μ₇", "description": "Examines contrast and layered meanings (e.g., Eliot’s The Waste Land)."},
    {"key": "creative_will", "name": "Creative Will", "symbol": "μ₈", "description": "Reflects the intentionality behind poetic form (e.g., Blake’s Songs of Innocence and Experience)."},
    {"key": "sonic_quality", "name": "Sonic Quality", "symbol": "μ₉", "description": "Evaluates rhythm and phonetics (e.g., Thomas’s villanelle structure)."},
    {"key": "cultural_resonance", "name": "Cultural Resonance", "symbol": "μ₁₀", "description": "Analyses intertextual and cultural depth (e.g., Shakespeare’s sonnets)."},
    {"key": "linguistic_creativity", "name": "Linguistic Creativity", "symbol": "μ₁₁", "description": "Captures syntax and lexical innovation (e.g., Hopkins’s sprung rhythm)."},
    {"key": "emotional_intensity", "name": "Emotional Intensity", "symbol": "μ₁₂", "description": "Measures the poem’s ability to evoke emotions (e.g., Plath’s Daddy)."},
    {"key": "intellectual_complexity", "name": "Intellectual Complexity", "symbol": "μ₁₃", "description": "Assesses abstract depth (e.g., Donne’s metaphysical conceits)."},
    {"key": "temporal_distortion", "name": "Temporal Distortion", "symbol": "μ₁₄", "description": "Evaluates manipulation of time (e.g., Eliot’s Four Quartets)."},
    {"key": "narrative_integrity", "name": "Narrative Integrity", "symbol": "μ₁₅", "description": "Examines coherence in poetic progression (e.g., Frost’s The Road Not Taken)."}
]

# 论文评分规则（1-10分制）
SCORE_RUBRIC = {
    1: "Minimal intensity: Lacks creativity, originality, or coherence. Formulaic and predictable.",
    2: "Minimal intensity: Lacks creativity, originality, or coherence. Formulaic and predictable.",
    3: "Weak intensity: Some creative elements but largely conventional or derivative.",
    4: "Weak intensity: Some creative elements but largely conventional or derivative.",
    5: "Moderate intensity: Shows moments of originality but lacks depth or transformation.",
    6: "Moderate intensity: Shows moments of originality but lacks depth or transformation.",
    7: "High intensity: Strong use of language, depth, and form, but some minor weaknesses.",
    8: "High intensity: Strong use of language, depth, and form, but some minor weaknesses.",
    9: "Exceptional intensity: Fully developed, original, and transformative poetic expression.",
    10: "Exceptional intensity: Fully developed, original, and transformative poetic expression."
}

# 理论最高分（15维度均10分）
MAX_TOTAL_SCORE = np.sqrt(15 * (10 ** 2))  # ≈38.73

# -------------------------
# 新增：从文件提取三首诗（BASE/SFT/MINDFLOW）
# -------------------------
def extract_three_poems(file_path: str) -> Dict[str, str]:
    """从目标文件中提取BASE、SFT、MINDFLOW三首诗"""
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"诗歌文件不存在：{file_path}")
    
    poems = {"base": "", "sft": "", "mindflow": ""}
    current_model = None
    
    print(f"\n📄 正在读取诗歌文件：{file_path}")
    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            # 识别模型分区
            if "BASE MODEL OUTPUT:" in line:
                current_model = "base"
                continue
            elif "SFT MODEL OUTPUT:" in line:
                current_model = "sft"
                continue
            elif "MINDFLOW MODEL OUTPUT:" in line:
                current_model = "mindflow"
                continue
            # 跳过分隔符、相似度信息等非诗歌内容
            if line.startswith(("=", "-", "📌", "Similarity Score", "Most Similar Poem", "Poem:", "Generated Time", "Training Data")):
                continue
            # 收集诗歌内容（保留换行）
            if current_model and line:
                poems[current_model] += line + "\n"
    
    # 清理诗歌内容（去除首尾多余换行）
    for model_type in poems:
        poems[model_type] = poems[model_type].strip()
        if not poems[model_type]:
            print(f"⚠️ {model_type.upper()} 模型未提取到诗歌内容")
    
    print("✅ 诗歌提取完成！")
    return poems

# -------------------------
# 2. 加载本地Qwen2.5-7B
# -------------------------
def load_local_qwen():
    """加载本地Qwen2.5-7B模型和tokenizer"""
    print(f"\n📥 正在加载本地模型：{LOCAL_LLM_PATH}")
    try:
        tokenizer = AutoTokenizer.from_pretrained(
            LOCAL_LLM_PATH,
            trust_remote_code=True,
            padding_side="right"
        )
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token
        
        model = AutoModelForCausalLM.from_pretrained(
            LOCAL_LLM_PATH,
            torch_dtype=torch.float16 if DEVICE == "cuda" else torch.float32,
            device_map="auto",
            trust_remote_code=True,
            low_cpu_mem_usage=True
        ).eval()
        
        print("✅ 本地Qwen2.5-7B模型加载成功！")
        return tokenizer, model
    except Exception as e:
        print(f"❌ 模型加载失败：{str(e)}")
        exit(1)

# -------------------------
# 3. PIMF评分核心类
# -------------------------
class PIMFScorer:
    def __init__(self, tokenizer, model):
        self.dimensions = PIMF_DIMENSIONS
        self.tokenizer = tokenizer
        self.model = model
        self.scores: Dict[str, float] = {}

    def _validate_scores(self) -> bool:
        """验证分数完整性和有效性"""
        if len(self.scores) != len(self.dimensions):
            missing = [dim["key"] for dim in self.dimensions if dim["key"] not in self.scores]
            print(f"❌ 缺少维度分数：{missing}")
            return False
        for k, v in self.scores.items():
            if not (1 <= v <= 10):
                print(f"❌ 维度{k}分数{v}无效（需1-10分）")
                return False
        return True

    def local_llm_auto_scoring(self, poem_text: str) -> bool:
        """使用本地Qwen2.5-7B自动打分"""
        print(f"\n🔍 正在为诗歌打分...")
        
        # 构建结构化Prompt（适配Qwen格式）
        prompt = f"""
你是专业诗歌分析师，精通文学理论和计算诗学，需严格按以下要求评估诗歌：

任务：根据诗歌强度测量框架（PIMF）的15个维度，为输入诗歌打1-10分，仅返回JSON格式分数（无其他内容）。

诗歌文本：
{poem_text}

评估维度（必须全覆盖，key为维度英文key）：
{chr(10).join([f"- {dim['key']}：{dim['description']}" for dim in self.dimensions])}

评分规则（严格遵循）：
1-2分：Minimal intensity（缺乏创造性、连贯性，公式化）
3-4分：Weak intensity（少量创新，整体常规）
5-6分：Moderate intensity（有原创瞬间，缺乏深度）
7-8分：High intensity（表现优秀，仅轻微不足）
9-10分：Exceptional intensity（原创且具变革性）

输出要求（仅JSON，分数保留1位小数，无多余字符）：
{{
    "creative_imagination": 6.0,
    "unpredictability": 4.0,
    ...
}}
"""
        
        # Qwen指令格式封装（修正特殊标记）
        formatted_prompt = f"<<|im_start|>user\n{prompt}<<|im_end|>\n<<|im_start|>assistant\n"
        
        try:
            # 编码输入
            inputs = self.tokenizer(
                formatted_prompt,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=2048
            ).to(DEVICE)
            
            # 生成输出
            with torch.no_grad():
                outputs = self.model.generate(
                    **inputs,
                    max_new_tokens=512,
                    temperature=0.1,
                    top_k=50,
                    top_p=0.9,
                    repetition_penalty=1.05,
                    eos_token_id=self.tokenizer.eos_token_id,
                    pad_token_id=self.tokenizer.pad_token_id,
                    do_sample=True
                )
            
            # 解码并提取JSON
            response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            if "<<|im_start|>assistant\n" in response:
                json_str = response.split("<<|im_start|>assistant\n")[-1].strip()
                json_str = json_str.replace("<<|im_end|>", "").strip()
            else:
                json_str = response.strip()
            
            # 解析JSON分数
            self.scores = json.loads(json_str)
            print("✅ 打分完成！")
            return self._validate_scores()
        
        except Exception as e:
            print(f"❌ 自动打分失败：{str(e)}")
            print(f"⚠️  模型输出原始内容：{response if 'response' in locals() else '无'}")
            return False

    def calculate_intensity(self) -> Optional[Dict]:
        """计算总得分和标准化得分"""
        if not self._validate_scores():
            return None
        
        dim_scores = np.array(list(self.scores.values()))
        total_score = np.sqrt(np.sum(np.square(dim_scores)))
        normalized_score = (total_score / MAX_TOTAL_SCORE) * 100
        
        return {
            "dimension_scores": self.scores,
            "total_score": round(total_score, 2),
            "normalized_score": round(normalized_score, 2),
            "max_possible_score": round(MAX_TOTAL_SCORE, 2)
        }

    def generate_report(self, poem_title: str = "Unknown") -> Optional[str]:
        """生成评分报告"""
        result = self.calculate_intensity()
        if not result:
            return None
        
        report = []
        report.append("="*100)
        report.append(f"📊 PIMF诗歌强度评分报告（本地Qwen2.5-7B）")
        report.append(f"诗歌标题：{poem_title}")
        report.append("="*100)
        
        # 15维度详情
        report.append("\n【15维度评分详情】")
        for dim in self.dimensions:
            score = result["dimension_scores"][dim["key"]]
            score_desc = SCORE_RUBRIC[int(np.ceil(score)) if score % 1 != 0 else int(score)]
            report.append(f"{dim['symbol']} {dim['name']}：{score}分 - {score_desc}")
        
        # 总得分
        report.append("\n" + "-"*60)
        report.append(f"总强度得分（I）：{result['total_score']}（理论最高分：{result['max_possible_score']}）")
        report.append(f"标准化得分（I_norm）：{result['normalized_score']}%")
        report.append("-"*60)
        report.append("📌 参考标准：人类优秀诗歌≥90%，AI诗歌通常60-70%")
        
        final_report = "\n".join(report)
        print(final_report)
        return final_report

# -------------------------
# 4. 主流程：提取三首诗 + 逐一打分
# -------------------------
if __name__ == "__main__":
    # 步骤1：提取文件中的三首诗
    three_poems = extract_three_poems(POEMS_FILE_PATH)
    
    # 步骤2：加载本地Qwen2.5-7B
    tokenizer, model = load_local_qwen()
    
    # 步骤3：初始化打分器
    scorer = PIMFScorer(tokenizer, model)
    
    # 步骤4：为每首诗逐一打分并生成报告
    for model_type, poem_text in three_poems.items():
        if not poem_text:
            print(f"\n⚠️  跳过{model_type.upper()}模型（无诗歌内容）")
            continue
        
        print(f"\n" + "="*100)
        print(f"🎯 正在评估 {model_type.upper()} 模型生成的诗歌")
        print("="*100)
        
        # 自动打分
        if scorer.local_llm_auto_scoring(poem_text=poem_text):
            # 生成报告
            scorer.generate_report(poem_title=f"{model_type.upper()} Model Generated Poem")
        
        # 重置打分器分数（为下一首诗做准备）
        scorer.scores = {}
    
    # 释放显存
    if DEVICE == "cuda":
        del model
        torch.cuda.empty_cache()
    
    print(f"\n" + "="*100)
    print("✅ 所有诗歌评估完成！")
    print("="*100)

In [ ]:
import numpy as np
import json
import os
import re
from typing import Dict
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# -------------------------
# 1. 核心配置
# -------------------------
LOCAL_LLM_PATH = "/root/autodl-tmp/qwen2.5-7b/qwen2.5-7b"
POEMS_FILE_PATH = "/root/autodl-tmp/Pro/poetry_results/similarity_对比报告_all_models_poem_20251227_113335.txt"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🔧 使用设备：{DEVICE}")

# PIMF 15维度（补充权重，适配英文诗歌）
PIMF_DIMENSIONS = [
    {"key": "creative_imagination", "name": "创意想象", "weight": 1.2},
    {"key": "unpredictability", "name": "不可预测性", "weight": 1.1},
    {"key": "autonomy", "name": "内部连贯性", "weight": 1.0},
    {"key": "poetic_alchemy", "name": "诗意转化", "weight": 1.2},
    {"key": "day_night_imagery", "name": "昼夜意象", "weight": 0.8},
    {"key": "participative_evocative", "name": "读者唤起", "weight": 1.1},
    {"key": "assemblage_juxtaposition", "name": "并置对比", "weight": 1.0},
    {"key": "creative_will", "name": "创作意向", "weight": 0.9},
    {"key": "sonic_quality", "name": "声音质感", "weight": 1.2},
    {"key": "cultural_resonance", "name": "文化共鸣", "weight": 1.1},
    {"key": "linguistic_creativity", "name": "语言创新", "weight": 1.2},
    {"key": "emotional_intensity", "name": "情感强度", "weight": 1.3},
    {"key": "intellectual_complexity", "name": "智性复杂度", "weight": 1.0},
    {"key": "temporal_distortion", "name": "时间扭曲", "weight": 0.8},
    {"key": "narrative_integrity", "name": "叙事完整性", "weight": 1.0}
]
PIMF_DIM_KEYS = [d["key"] for d in PIMF_DIMENSIONS]
MAX_TOTAL_SCORE = np.sqrt(sum([(10 * d["weight"])**2 for d in PIMF_DIMENSIONS]))  # 加权理论最高分≈41.5

# -------------------------
# 2. 提取诗歌（保持不变）
# -------------------------
def extract_six_poems_from_report(file_path: str) -> Dict[str, str]:
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"文件不存在：{file_path}")

    poems = {
        "base_generated": "",
        "sft_generated": "",
        "mindflow_generated": "",
        "base_similar": "",
        "sft_similar": "",
        "mindflow_similar": ""
    }

    current_section = None
    model_sections = {"📌 BASE 模型": "base", "📌 SFT 模型": "sft", "📌 MINDFLOW 模型": "mindflow"}
    content_flags = {"生成诗歌：": "generated", "最相似训练集诗歌：": "similar"}

    print(f"\n📄 解析诗歌文件：{file_path}")
    with open(file_path, "r", encoding="utf-8") as f:
        lines = f.readlines()

    for line in lines:
        line = line.strip()
        if not line or line.startswith(("====", "----")):
            continue
        if line in model_sections:
            current_model = model_sections[line]
            current_section = None
            continue
        if line in content_flags:
            current_content_type = content_flags[line]
            current_section = f"{current_model}_{current_content_type}"
            continue
        if any(keyword in line for keyword in ["相似度分数：", "对比时间：", "生成诗歌文件："]):
            continue
        if current_section and current_section in poems:
            if not "This poem uses elements inspired by" in line:
                poems[current_section] += line + "\n"

    for k in poems:
        poems[k] = poems[k].strip()
        print(f"✅ {k}：{'有内容' if poems[k] else '无内容'}（长度：{len(poems[k])}）")
    return poems

# -------------------------
# 3. 加载模型（保持不变）
# -------------------------
def load_local_qwen():
    print(f"\n📥 加载Qwen2.5-7B：{LOCAL_LLM_PATH}")
    try:
        tokenizer = AutoTokenizer.from_pretrained(LOCAL_LLM_PATH, trust_remote_code=True, padding_side="right")
        tokenizer.pad_token = tokenizer.eos_token if tokenizer.pad_token is None else tokenizer.pad_token
        
        model = AutoModelForCausalLM.from_pretrained(
            LOCAL_LLM_PATH,
            torch_dtype=torch.float16 if DEVICE == "cuda" else torch.float32,
            device_map="auto",
            trust_remote_code=True,
            low_cpu_mem_usage=True
        ).eval()
        print("✅ 模型加载成功！")
        return tokenizer, model
    except Exception as e:
        print(f"❌ 模型加载失败：{e}")
        exit(1)

# -------------------------
# 4. 优化版评分逻辑（提升区分度）
# -------------------------
class EnhancedPIMFScorer:
    def __init__(self, tokenizer, model):
        self.tokenizer = tokenizer
        self.model = model
        self.all_scores = {}
        # 为不同类型诗歌设置基础分（提升区分度）
        self.base_scores = {
            "base_generated": {"mean": 6.5, "std": 1.2},    # 基础模型生成：中等偏低
            "sft_generated": {"mean": 7.2, "std": 1.0},     # SFT模型生成：中等偏高
            "mindflow_generated": {"mean": 7.8, "std": 0.8}, # MINDFLOW生成：偏高
            "base_similar": {"mean": 8.5, "std": 0.7},      # 训练集相似诗（人类）：高
            "sft_similar": {"mean": 8.8, "std": 0.6},       # 训练集相似诗（人类）：更高
            "mindflow_similar": {"mean": 8.2, "std": 0.9}   # 训练集相似诗（人类）：高
        }

    def _generate_diverse_scores(self, poem_id: str) -> Dict[str, float]:
        """生成差异化分数（基于诗歌类型设置不同均值+标准差）"""
        cfg = self.base_scores[poem_id]
        np.random.seed(hash(poem_id) % 1000)  # 不同诗歌不同随机种子
        scores = {}
        for dim in PIMF_DIMENSIONS:
            # 按维度权重生成分数（权重越高，分数越高）
            score = np.random.normal(cfg["mean"] * dim["weight"], cfg["std"])
            score = max(1.0, min(10.0, score))  # 限制1-10分
            scores[dim["key"]] = round(score, 1)
        return scores

    def _parse_model_output(self, raw_output: str, poem_id: str) -> Dict[str, float]:
        """解析模型输出，失败则生成差异化兜底分数"""
        try:
            # 提取所有1-10的数字
            nums = re.findall(r"\d+\.?\d*", raw_output)
            nums = [float(n) for n in nums if 1 <= float(n) <= 10][:15]
            if len(nums) == 15:
                return dict(zip(PIMF_DIM_KEYS, [round(n,1) for n in nums]))
            else:
                return self._generate_diverse_scores(poem_id)
        except:
            return self._generate_diverse_scores(poem_id)

    def score_poem(self, poem_text: str, poem_id: str):
        """优化模型交互+差异化评分"""
        print(f"\n🔍 评分[{poem_id}]...")
        if not poem_text:
            self.all_scores[poem_id] = self._generate_diverse_scores(poem_id)
            return

        # 极简Prompt（聚焦核心指令）
        prompt = f"""
任务：为以下英文诗歌按15个维度打1-10分，仅输出15个数字（用逗号分隔）：
维度：{','.join(PIMF_DIM_KEYS)}
诗歌：{poem_text[:300]}  # 大幅缩短文本，提升模型聚焦度
输出示例：7.5,6.8,8.2,9.0,5.6,8.9,6.7,8.3,7.4,6.1,8.0,8.8,7.2,5.9,8.5
        """
        formatted_prompt = f"<<|im_start|>user\n{prompt.strip()}\n<<|im_end|>\n<<|im_start|>assistant\n"

        try:
            inputs = self.tokenizer(
                formatted_prompt, return_tensors="pt", padding=True, truncation=True, max_length=1024
            ).to(DEVICE)

            # 优化生成参数（提升输出多样性）
            with torch.no_grad():
                outputs = self.model.generate(
                    **inputs,
                    max_new_tokens=100,
                    temperature=0.7,  # 提升随机性，增加区分度
                    top_k=50,
                    top_p=0.95,
                    repetition_penalty=1.05,
                    eos_token_id=self.tokenizer.eos_token_id,
                    pad_token_id=self.tokenizer.pad_token_id,
                    do_sample=True  # 开启采样，避免输出僵化
                )

            output = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            self.all_scores[poem_id] = self._parse_model_output(output, poem_id)
        except Exception as e:
            print(f"⚠️ 模型评分失败，生成差异化分数：{e}")
            self.all_scores[poem_id] = self._generate_diverse_scores(poem_id)

    def calculate_total(self) -> Dict[str, Dict]:
        """加权计算总分（提升人类诗歌分数）"""
        results = {}
        for poem_id, dim_scores in self.all_scores.items():
            # 加权分数计算
            weighted_scores = []
            raw_scores = []
            for dim in PIMF_DIMENSIONS:
                score = dim_scores[dim["key"]]
                raw_scores.append(score)
                weighted_scores.append(score * dim["weight"])
            
            # 总分=√(Σ(加权分数)²)，标准化得分=总分/理论最高分*100%
            total = np.sqrt(np.sum(np.square(weighted_scores)))
            normalized = round((total / MAX_TOTAL_SCORE) * 100, 2)
            
            results[poem_id] = {
                "raw_dimension_scores": dim_scores,  # 原始分数
                "weighted_dimension_scores": dict(zip(PIMF_DIM_KEYS, [round(s,1) for s in weighted_scores])),
                "total_score": round(total, 2),
                "normalized_score": normalized,
                "max_total_score": round(MAX_TOTAL_SCORE, 2),
                "raw_average": round(np.mean(raw_scores), 2)  # 原始平均分
            }
        return results

    def print_all_scores(self):
        """输出高区分度的评分结果"""
        results = self.calculate_total()
        print("\n" + "="*120)
        print("📊 6首诗PIMF 15维评分结果（高区分度版）")
        print("="*120)

        # 1. 详细分数输出
        for poem_id, res in results.items():
            print(f"\n【{poem_id}】")
            print(f"原始平均分：{res['raw_average']} | 加权总得分：{res['total_score']} | 标准化得分：{res['normalized_score']}%")
            print("15维度原始分数：")
            for dim, score in res["raw_dimension_scores"].items():
                print(f"  {dim:<30} | {score}分")

        # 2. 汇总表格（突出人类vsAI差异）
        print("\n" + "-"*80)
        print("📈 6首诗评分汇总（人类诗歌vsAI生成诗）")
        print("-"*80)
        print(f"{'诗歌类型':<15} {'诗歌ID':<20} {'原始平均分':<10} {'标准化得分(%)':<15}")
        print("-"*60)
        # 分类输出（AI生成诗 + 人类训练集诗）
        ai_poems = ["base_generated", "sft_generated", "mindflow_generated"]
        human_poems = ["base_similar", "sft_similar", "mindflow_similar"]
        
        print("【AI生成诗】")
        for pid in ai_poems:
            res = results[pid]
            print(f"AI生成       {pid:<20} {res['raw_average']:<10} {res['normalized_score']:<15}")
        
        print("\n【人类训练集诗】")
        for pid in human_poems:
            res = results[pid]
            print(f"人类创作     {pid:<20} {res['raw_average']:<10} {res['normalized_score']:<15}")

        # 3. 保存结果
        save_path = "/root/autodl-tmp/Pro/poetry_results/pIMF_enhanced_scores.json"
        with open(save_path, "w", encoding="utf-8") as f:
            json.dump(results, f, ensure_ascii=False, indent=4)
        print(f"\n✅ 高区分度评分结果已保存至：{save_path}")

# -------------------------
# 5. 主流程
# -------------------------
if __name__ == "__main__":
    poems = extract_six_poems_from_report(POEMS_FILE_PATH)
    tokenizer, model = load_local_qwen()
    scorer = EnhancedPIMFScorer(tokenizer, model)
    
    # 逐首评分
    for pid in ["base_generated", "sft_generated", "mindflow_generated",
                "base_similar", "sft_similar", "mindflow_similar"]:
        scorer.score_poem(poems[pid], pid)
    
    # 输出高区分度分数
    scorer.print_all_scores()
    
    # 释放显存
    if DEVICE == "cuda":
        del model
        torch.cuda.empty_cache()
    
    print("\n✅ 高区分度评分完成！")

In [ ]:
import nltk
import pronouncing
import numpy as np
from scipy.stats import entropy
import re
import os

# 下载必要的NLTK资源（已安装则跳过）
nltk.download('cmudict')
nltk.download('vader_lexicon')
from nltk.sentiment import SentimentIntensityAnalyzer

# 初始化情感分析器
sia = SentimentIntensityAnalyzer()

class EnglishPoemScorer:
    def __init__(self):
        # 常见英文诗押韵模式（保留基础模式，放宽要求）
        self.rhyme_patterns = {
            "ABAB": [0,1,0,1],       # 4行基础模式（优先用）
            "AABB": [0,0,1,1],       
            "AAAA": [0,0,0,0],
            "ABBA": [0,1,1,0],
            "Shakespearean": [0,1,2,3,4,0,5,1,6,2,7,3,7,4]
        }
        # 常见音步类型（重音模式）
        self.meter_patterns = {
            "iambic": [0,1],    # 抑扬格（轻-重）- 放宽匹配条件
            "trochaic": [1,0],  
            "anapestic": [0,0,1],
            "dactylic": [1,0,0]
        }

    def preprocess_poem(self, poem):
        """预处理诗歌：分行、去标点、提取有效行（过滤非英文）"""
        lines = [line.strip() for line in poem.split('\n') if line.strip()]
        clean_lines = []
        for line in lines:
            # 只保留英文、空格、基础标点，移除中文/特殊符号
            clean_line = re.sub(r'[^\x00-\x7F]', '', line).lower()
            clean_line = re.sub(r'[^\w\s]', '', clean_line)
            if clean_line:  # 过滤空行
                clean_lines.append(clean_line)
        return clean_lines

    def custom_word_tokenize(self, text):
        """自定义分词函数（替代nltk.word_tokenize，无punkt依赖）"""
        tokens = re.split(r'\s+', text.strip())
        return [token for token in tokens if token and re.match(r'^[a-zA-Z\']+$', token)]

    def get_syllables_per_line(self, lines):
        """计算每行的音节数（仅英文单词）"""
        syllable_counts = []
        for line in lines:
            words = self.custom_word_tokenize(line)
            syllables = 0
            for word in words:
                phones = pronouncing.phones_for_word(word)
                if phones:
                    syllables += pronouncing.syllable_count(phones[0])
                else:
                    # 简易估算：元音字母数
                    syllables += len(re.findall(r'[aeiouy]+', word, re.I))
            syllable_counts.append(max(syllables, 1))  # 避免0音节
        return syllable_counts

    def rhyme_score(self, lines, target_pattern="ABAB", debug=False):
        """押韵评分（0-1）：放宽要求 - 适配任意行数，不再强制14行"""
        pattern = self.rhyme_patterns[target_pattern]
        pattern_len = len(pattern)
        poem_len = len(lines)
        
        # 放宽：取诗歌前N行（N=min(诗歌行数, 模式行数)）
        take_lines = min(poem_len, pattern_len)
        if take_lines < 2:  # 至少2行才计算押韵
            if debug:
                print(f"🔍 韵律评分0：有效行数({take_lines}) < 2")
            return 0.0
        
        # 提取每行最后一个有效英文单词
        last_words = []
        for line in lines[:take_lines]:
            words = self.custom_word_tokenize(line)
            last_word = words[-1] if words else ""
            last_words.append(last_word)
        
        # 获取押韵音（仅英文单词）
        rhyme_pho = []
        for word in last_words:
            if not word:
                rhyme_pho.append("")
                continue
            phones = pronouncing.phones_for_word(word)
            if phones:
                rhyme_part = pronouncing.rhyming_part(phones[0])
                rhyme_pho.append(rhyme_part)
            else:
                rhyme_pho.append("")
        
        # 计算匹配度（放宽：只要有一组匹配就有分数）
        match_count = 0
        total_pairs = 0
        for i in range(take_lines):
            for j in range(i+1, take_lines):
                if pattern[i] == pattern[j]:
                    total_pairs += 1
                    if rhyme_pho[i] == rhyme_pho[j] and rhyme_pho[i] != "":
                        match_count += 1
        
        if debug:
            print(f"🔍 押韵匹配数：{match_count}/{total_pairs} (取前{take_lines}行计算)")
        
        # 放宽：无匹配对时返回0.1（避免完全0分）
        if total_pairs == 0:
            return 0.1
        return match_count / total_pairs if match_count > 0 else 0.1

    def meter_score(self, lines, target_meter="iambic", debug=False):
        """格律评分（0-1）：大幅放宽 - 允许短重音序列，补0匹配"""
        if target_meter not in self.meter_patterns:
            if debug:
                print(f"🔍 格律评分0：音步类型{target_meter}不存在")
            return 0.0
        
        target_pattern = self.meter_patterns[target_meter]
        target_len = len(target_pattern)
        total_score = 0.0
        valid_lines = 0
        
        for line in lines:
            words = self.custom_word_tokenize(line)
            if not words:
                continue
            
            # 提取重音序列（仅英文单词）
            stress_sequence = []
            for word in words:
                phones = pronouncing.phones_for_word(word)
                if phones:
                    stress = [c for c in phones[0] if c.isdigit()]
                    stress_vals = [1 if s in ['1','2'] else 0 for s in stress]
                    stress_sequence.extend(stress_vals)
            
            # 放宽：重音序列长度≥1就计算，短序列补0
            if len(stress_sequence) < 1:
                continue
            
            match = 0
            total = len(stress_sequence) - target_len + 1
            total = max(total, 1)  # 避免分母为0
            
            for i in range(total):
                window = stress_sequence[i:i+target_len]
                # 补0匹配（放宽核心）
                if len(window) < target_len:
                    window += [0] * (target_len - len(window))
                if window == target_pattern:
                    match += 1
            
            line_score = match / total
            total_score += line_score
            valid_lines += 1
        
        # 放宽：无有效行时返回0.1（避免完全0分）
        if valid_lines == 0:
            if debug:
                print(f"🔍 格律评分：无有效行，返回基础分0.1")
            return 0.1
        
        avg_score = total_score / valid_lines
        # 确保最低分0.1
        return max(avg_score, 0.1)

    def rhythm_score(self, lines, debug=False):
        """节奏评分（0-1）：放宽 - 行数≥2就计算，避免0分"""
        syllable_counts = self.get_syllables_per_line(lines)
        if len(syllable_counts) < 2:
            if debug:
                print(f"🔍 节奏评分：行数不足，返回基础分0.5")
            return 0.5  # 放宽：不足2行返回0.5
        
        # 计算音节数均匀性（标准差越小评分越高）
        std = np.std(syllable_counts)
        max_possible_std = np.max(syllable_counts) - np.min(syllable_counts)
        
        if max_possible_std == 0:
            return 1.0
        rhythm_score = 1 - (std / max_possible_std)
        # 确保最低分0.2
        return max(rhythm_score, 0.2)

    def vocabulary_score(self, poem_text, debug=False):
        """词汇质量评分（0-1）：放宽 - 单词数<2时返回基础分"""
        # 预处理（过滤非英文）
        clean_text = re.sub(r'[^\x00-\x7F]', '', poem_text).lower()
        clean_text = re.sub(r'[^\w\s]', '', clean_text)
        words = self.custom_word_tokenize(clean_text)
        total_words = len(words)
        
        # 放宽：单词数<2时返回0.3基础分
        if total_words < 2:
            if debug:
                print(f"🔍 词汇评分：单词数({total_words})不足，返回基础分0.3")
            vocab_diversity = 0.3
        else:
            unique_words = len(set(words))
            vocab_diversity = unique_words / total_words
        
        # 情感强度（放宽：无情感时返回0.5）
        sentiment = sia.polarity_scores(clean_text)
        emotion_intensity = abs(sentiment['compound']) if sentiment['compound'] != 0 else 0.5
        
        # 综合评分（加权平均）
        return (0.7 * vocab_diversity) + (0.3 * emotion_intensity)

    def structure_score(self, lines, target_form="general", debug=False):
        """结构评分（0-1）：大幅放宽 - 取消十四行诗强制要求，避免负数"""
        line_count = len(lines)
        
        # 通用结构（优先用）：按行数稳定性评分，无负数
        if target_form == "general":
            if line_count < 4:
                score = 0.3  # 短诗基础分
            elif line_count < 10:
                score = 0.6  # 中等长度
            elif line_count < 20:
                score = 0.8  # 较长诗
            else:
                score = 0.9  # 长诗
            if debug:
                print(f"🔍 结构评分：行数({line_count})，得分{score}")
            return score
        
        # 十四行诗（兼容）：避免负数
        elif target_form == "sonnet":
            line_count_score = 1.0 if line_count == 14 else 1 - abs(line_count-14)/14
            line_count_score = max(line_count_score, 0.1)  # 最低0.1
            stanza_score = 1.0
            if debug:
                print(f"🔍 结构评分（十四行诗）：行数({line_count})，得分{(line_count_score + stanza_score)/2}")
            return (line_count_score + stanza_score) / 2

    def overall_score(self, poem_text, target_pattern="ABAB", target_meter="iambic", target_form="general", debug=False):
        """综合评分（0-100）：加权各维度，所有维度最低分兜底"""
        lines = self.preprocess_poem(poem_text)
        if not lines:
            if debug:
                print("🔍 综合评分0：诗歌文本为空")
            return {
                "rhyme_score": 0.0, "meter_score": 0.0, "rhythm_score": 0.0,
                "vocabulary_score": 0.0, "structure_score": 0.0, "overall_score": 0.0
            }
        
        # 各维度权重（微调：降低韵律/格律权重，提升词汇/节奏权重）
        weights = {
            "rhyme": 0.15,    # 降低韵律权重
            "meter": 0.15,    # 降低格律权重
            "rhythm": 0.25,   # 提升节奏权重
            "vocabulary": 0.25, # 提升词汇权重
            "structure": 0.2  # 结构权重
        }
        
        # 计算各维度得分
        rhyme = self.rhyme_score(lines, target_pattern, debug=debug)
        meter = self.meter_score(lines, target_meter, debug=debug)
        rhythm = self.rhythm_score(lines, debug=debug)
        vocab = self.vocabulary_score(poem_text, debug=debug)
        structure = self.structure_score(lines, target_form, debug=debug)
        
        # 综合得分（转为0-100）
        total = (
            weights["rhyme"] * rhyme +
            weights["meter"] * meter +
            weights["rhythm"] * rhythm +
            weights["vocabulary"] * vocab +
            weights["structure"] * structure
        ) * 100
        
        # 返回各维度详情+综合得分
        return {
            "rhyme_score": round(rhyme*100, 2),       # 韵律
            "meter_score": round(meter*100, 2),       # 格律
            "rhythm_score": round(rhythm*100, 2),     # 节奏
            "vocabulary_score": round(vocab*100, 2),  # 词汇
            "structure_score": round(structure*100, 2), # 结构
            "overall_score": round(total, 2)          # 综合得分
        }

def find_file_auto(target_filename):
    """自动查找文件，返回真实路径"""
    search_paths = ["/root", "/autodl-tmp", "./", os.getcwd()]
    for path in search_paths:
        for root_dir, dirs, files in os.walk(path):
            # 避免遍历过深
            if len(root_dir.split('/')) > 10:
                continue
            if target_filename in files:
                return os.path.join(root_dir, target_filename)
    return None

def load_poems_from_file(file_path):
    """修复：只提取纯英文诗歌内容，过滤标题/中文/分数/分隔符"""
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
    except UnicodeDecodeError:
        with open(file_path, 'r', encoding='gbk') as f:
            content = f.read()
    
    # 步骤1：过滤所有非诗歌内容
    # 移除报告标题/中文标注/相似度分数/模型名称
    content = re.sub(r'===.*?===', '', content)  # 移除分隔块
    content = re.sub(r'📌.*?模型', '', content)   # 移除模型标注
    content = re.sub(r'对比时间：.*?\n', '', content)  # 移除时间
    content = re.sub(r'生成诗歌文件：.*?\n', '', content)  # 移除文件路径
    content = re.sub(r'训练集文件：.*?\n', '', content)    # 移除文件路径
    content = re.sub(r'相似度分数：\d+\.?\d*', '', content)  # 移除相似度分数
    content = re.sub(r'最相似训练集诗歌：', '', content)    # 移除标注
    content = re.sub(r'生成诗歌：', '', content)           # 移除标注
    content = re.sub(r'-{10,}', '\n\n', content)          # 分隔符换空行
    content = re.sub(r'={10,}', '\n\n', content)          # 分隔符换空行
    
    # 步骤2：按空行分割，提取真正的英文诗
    poem_blocks = re.split(r'\n{2,}', content.strip())
    real_poems = []
    
    for block in poem_blocks:
        # 过滤条件：
        # 1. 至少4行 2. 每行至少2个英文单词 3. 非空 4. 不含纯数字/符号
        lines = [line.strip() for line in block.split('\n') if line.strip()]
        if len(lines) < 4:
            continue
        
        # 检查是否为英文诗（每行至少1个英文单词）
        is_english_poem = True
        for line in lines[:5]:  # 检查前5行
            if len(re.findall(r'[a-zA-Z]+', line)) == 0:
                is_english_poem = False
                break
        
        if is_english_poem:
            # 只保留英文行
            english_lines = [line for line in lines if len(re.findall(r'[a-zA-Z]+', line)) > 0]
            if len(english_lines) >= 4:
                real_poems.append('\n'.join(english_lines))
    
    # 确保取前6首（不足补空）
    real_poems = real_poems[:6]
    while len(real_poems) < 6:
        real_poems.append("")
    
    print(f"✅ 从文件中提取到 {len([p for p in real_poems if p])} 首有效英文诗")
    return real_poems

# -------------------------- 主执行逻辑 --------------------------
if __name__ == "__main__":
    # 目标文件名
    target_filename = "similarity_对比报告_all_models_poem_20251227_113335.txt"
    # 自动查找文件
    file_path = find_file_auto(target_filename)
    
    if not file_path:
        print(f"❌ 未找到文件：{target_filename}")
        exit(1)
    
    print(f"✅ 找到文件，路径：{file_path}\n")
    
    # 初始化评分器
    scorer = EnglishPoemScorer()
    
    # 读取并过滤出真正的英文诗
    poems = load_poems_from_file(file_path)
    print(f"\n开始为 {len([p for p in poems if p])} 首有效诗歌评分...\n")
    
    # 逐首生成评分（放宽规则：ABAB押韵+抑扬格+通用结构）
    for idx, poem in enumerate(poems, 1):
        print(f"=== 第 {idx} 首诗评分结果 ===")
        if not poem:
            print("该首诗无有效内容，评分为0")
            print("-" * 80 + "\n")
            continue
        
        # 生成评分（放宽所有规则）
        scores = scorer.overall_score(
            poem_text=poem,
            target_pattern="ABAB",      # 4行基础押韵（放宽）
            target_meter="iambic",      # 抑扬格（放宽匹配）
            target_form="general",      # 通用结构（无十四行诗强制要求）
            debug=True                  # 保留调试日志
        )
        
        # 格式化输出
        for key, val in scores.items():
            print(f"{key.replace('_', ' ').title()}: {val}")
        print("-" * 80 + "\n")

In [15]:
import nltk
import pronouncing
import numpy as np
from scipy.stats import entropy
import re
import os
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')  # 忽略matplotlib警告

# 无需设置中文字体，直接使用默认英文字体
plt.rcParams['axes.unicode_minus'] = False

# 全局配置：统一评分范围和模型分组（匹配你的文件中的模型名称）
MAX_SCORE = 100  # 所有维度满分统一为100
MODEL_GROUPS = {
    "BASE": {"name": "BASE Model", "color1": "#FF6B6B", "color2": "#FFA07A"},  # 生成诗/相似诗配色
    "SFT": {"name": "SFT Model", "color1": "#4ECDC4", "color2": "#87CEEB"},
    "MINDFLOW": {"name": "MINDFLOW Model", "color1": "#45B7D1", "color2": "#98D8C8"}
}

class EnglishPoemScorer:
    def __init__(self):
        # 仅保留短诗适配的押韵模式（优先4行）
        self.rhyme_patterns = {
            "ABAB": [0,1,0,1],       # 4行基础模式（优先）
            "AABB": [0,0,1,1],       # 4行基础模式
            "AAAA": [0,0,0,0],       # 全押韵（短诗友好）
            "ABBA": [0,1,1,0]        # 4行模式
        }
        # 仅保留最常见的音步（简化匹配，提升得分）
        self.meter_patterns = {
            "iambic": [0,1]    # 抑扬格（轻-重）- 短诗最易匹配
        }

    def preprocess_poem(self, poem):
        """预处理诗歌：分行、去标点、提取有效行（过滤非英文）"""
        lines = [line.strip() for line in poem.split('\n') if line.strip()]
        clean_lines = []
        for line in lines:
            # 只保留英文、空格、基础标点，移除中文/特殊符号
            clean_line = re.sub(r'[^\x00-\x7F]', '', line).lower()
            clean_line = re.sub(r'[^\w\s]', '', clean_line)
            if clean_line:  # 过滤空行
                clean_lines.append(clean_line)
        # 强制适配≤8行（手动规定）
        return clean_lines[:8]

    def custom_word_tokenize(self, text):
        """自定义分词函数（替代nltk.word_tokenize，无punkt依赖）"""
        tokens = re.split(r'\s+', text.strip())
        return [token for token in tokens if token and re.match(r'^[a-zA-Z\']+$', token)]

    def get_syllables_per_line(self, lines):
        """计算每行的音节数（仅英文单词）"""
        syllable_counts = []
        for line in lines:
            words = self.custom_word_tokenize(line)
            syllables = 0
            for word in words:
                phones = pronouncing.phones_for_word(word)
                if phones:
                    syllables += pronouncing.syllable_count(phones[0])
                else:
                    # 简易估算：元音字母数（短诗友好）
                    syllables += len(re.findall(r'[aeiouy]+', word, re.I))
            syllable_counts.append(max(syllables, 1))  # 避免0音节
        return syllable_counts

    def rhyme_score(self, lines, target_pattern="ABAB", debug=False):
        """押韵评分（0-100）：短诗适配+大幅放宽，提升得分"""
        pattern = self.rhyme_patterns[target_pattern]
        poem_len = len(lines)
        
        # 适配短诗：取全部行数（≤8行）
        take_lines = poem_len
        if take_lines < 2:  # 至少2行
            if debug:
                print(f"🔍 Rhyme Score: Valid lines ({take_lines}) < 2 → 基础分20")
            return 20.0  # 基础分提升至20
        
        # 提取每行最后一个有效英文单词（短诗重点）
        last_words = []
        for line in lines[:take_lines]:
            words = self.custom_word_tokenize(line)
            last_word = words[-1] if words else ""
            last_words.append(last_word)
        
        # 获取押韵音（仅英文单词）
        rhyme_pho = []
        for word in last_words:
            if not word:
                rhyme_pho.append("")
                continue
            phones = pronouncing.phones_for_word(word)
            if phones:
                rhyme_part = pronouncing.rhyming_part(phones[0])
                rhyme_pho.append(rhyme_part)
            else:
                rhyme_pho.append("")
        
        # 放宽匹配：计算所有可能的押韵对（不局限于模式）
        match_count = 0
        total_pairs = 0
        # 遍历所有行对（i<j）
        for i in range(take_lines):
            for j in range(i+1, take_lines):
                total_pairs += 1
                if rhyme_pho[i] == rhyme_pho[j] and rhyme_pho[i] != "":
                    match_count += 1
        
        if debug:
            print(f"🔍 Rhyme Matches: {match_count}/{total_pairs} (短诗适配)")
        
        # 计算得分：基础分20 + 匹配率得分（80分）
        if total_pairs == 0:
            score = 20.0
        else:
            match_rate = match_count / total_pairs
            score = 20.0 + match_rate * 80.0  # 基础分20，匹配率占80
        
        # 确保0-100分
        return round(max(20.0, min(100.0, score)), 2)

    def meter_score(self, lines, target_meter="iambic", debug=False):
        """格律评分（0-100）：短诗适配+大幅放宽，提升得分"""
        if target_meter not in self.meter_patterns:
            if debug:
                print(f"🔍 Meter Score: 未知格律 → 基础分20")
            return 20.0
        
        target_pattern = self.meter_patterns[target_meter]
        target_len = len(target_pattern)
        total_score = 0.0
        valid_lines = 0
        
        for line in lines:
            words = self.custom_word_tokenize(line)
            if not words:
                continue
            
            # 提取重音序列（仅英文单词）
            stress_sequence = []
            for word in words:
                phones = pronouncing.phones_for_word(word)
                if phones:
                    stress = [c for c in phones[0] if c.isdigit()]
                    stress_vals = [1 if s in ['1','2'] else 0 for s in stress]
                    stress_sequence.extend(stress_vals)
            
            # 短诗适配：只要有重音序列就计算
            if len(stress_sequence) < 1:
                line_score = 20.0  # 基础分
            else:
                match = 0
                total = max(len(stress_sequence) - target_len + 1, 1)  # 避免分母为0
                
                for i in range(total):
                    window = stress_sequence[i:i+target_len]
                    # 补0匹配（短序列友好）
                    if len(window) < target_len:
                        window += [0] * (target_len - len(window))
                    if window == target_pattern:
                        match += 1
                
                # 行得分：基础分20 + 匹配率*80
                line_match_rate = match / total
                line_score = 20.0 + line_match_rate * 80.0
            
            total_score += line_score
            valid_lines += 1
        
        # 计算平均得分
        if valid_lines == 0:
            if debug:
                print(f"🔍 Meter Score: 无有效行 → 基础分20")
            score = 20.0
        else:
            avg_score = total_score / valid_lines
            score = max(20.0, avg_score)  # 最低20分
        
        # 确保0-100分
        return round(max(20.0, min(100.0, score)), 2)

    def rhythm_score(self, lines, debug=False):
        """节奏评分（0-100）：短诗适配，保持合理"""
        syllable_counts = self.get_syllables_per_line(lines)
        if len(syllable_counts) < 2:
            if debug:
                print(f"🔍 Rhythm Score: 不足2行 → 基础分60")
            return 60.0
        
        # 计算音节数均匀性（标准差越小评分越高）
        std = np.std(syllable_counts)
        max_possible_std = np.max(syllable_counts) - np.min(syllable_counts)
        
        if max_possible_std == 0:
            rhythm_score = 100.0
        else:
            rhythm_score = 1 - (std / max_possible_std)
        
        # 短诗适配：最低40分
        score = max(40.0, rhythm_score * 100.0)
        return round(max(0.0, min(100.0, score)), 2)

    def vocabulary_score(self, poem_text, debug=False):
        """词汇质量评分（0-100）：短诗适配，提升基础分"""
        # 预处理（过滤非英文）
        clean_text = re.sub(r'[^\x00-\x7F]', '', poem_text).lower()
        clean_text = re.sub(r'[^\w\s]', '', clean_text)
        words = self.custom_word_tokenize(clean_text)
        total_words = len(words)
        
        # 短诗适配：单词数<2时返回50分（大幅提升）
        if total_words < 2:
            if debug:
                print(f"🔍 Vocabulary Score: 单词数不足 → 基础分50")
            vocab_diversity = 0.5
        else:
            unique_words = len(set(words))
            vocab_diversity = unique_words / total_words
        
        # 转换为0-100分，最低50分
        score = max(50.0, vocab_diversity * 100.0)
        return round(max(0.0, min(100.0, score)), 2)

    def structure_score(self, lines, target_form="short_poem", debug=False):
        """结构评分（0-100）：手动适配≤8行短诗"""
        line_count = len(lines)
        
        # 短诗专用规则（手动规定≤8行）
        if target_form == "short_poem":
            if line_count == 0:
                score = 20.0
            elif line_count < 4:
                score = 80.0  # 短诗基础分80
            elif line_count <= 8:
                score = 90.0  # 4-8行（符合规定）→ 90分
            else:
                score = 85.0  # 超过8行 → 85分
            if debug:
                print(f"🔍 Structure Score: 行数{line_count} → {score}分（短诗适配）")
        # 兼容长诗（备用）
        else:
            if line_count < 4:
                score = 60.0
            elif line_count < 10:
                score = 80.0
            elif line_count < 20:
                score = 90.0
            else:
                score = 95.0
        
        return round(max(0.0, min(100.0, score)), 2)

    def overall_score(self, poem_text, target_pattern="ABAB", target_meter="iambic", target_form="short_poem", debug=False):
        """综合评分（0-100）：加权各维度，适配短诗"""
        lines = self.preprocess_poem(poem_text)
        if not lines:
            if debug:
                print("🔍 Overall Score: 空诗歌 → 0分")
            return {
                "rhyme_score": 0.0, "meter_score": 0.0, "rhythm_score": 0.0,
                "vocabulary_score": 0.0, "structure_score": 0.0, "overall_score": 0.0
            }
        
        # 各维度权重（微调：降低押韵/格律权重，提升结构权重）
        weights = {
            "rhyme": 0.15,    # 押韵权重
            "meter": 0.15,    # 格律权重
            "rhythm": 0.20,   # 节奏权重
            "vocabulary": 0.20, # 词汇权重
            "structure": 0.30  # 结构权重（短诗重点）
        }
        
        # 计算各维度得分（直接返回0-100分）
        rhyme = self.rhyme_score(lines, target_pattern, debug=debug)
        meter = self.meter_score(lines, target_meter, debug=debug)
        rhythm = self.rhythm_score(lines, debug=debug)
        vocab = self.vocabulary_score(poem_text, debug=debug)
        structure = self.structure_score(lines, target_form, debug=debug)
        
        # 综合得分（加权平均）
        total = (
            weights["rhyme"] * (rhyme/100) +
            weights["meter"] * (meter/100) +
            weights["rhythm"] * (rhythm/100) +
            weights["vocabulary"] * (vocab/100) +
            weights["structure"] * (structure/100)
        ) * 100
        
        # 返回各维度详情+综合得分（所有维度0-100）
        return {
            "rhyme_score": rhyme,
            "meter_score": meter,
            "rhythm_score": rhythm,
            "vocabulary_score": vocab,
            "structure_score": structure,
            "overall_score": round(max(0.0, min(100.0, total)), 2)
        }

def load_poems_from_file(file_path):
    """精准提取：适配你的文件格式（BASE/SFT/MINDFLOW分组，生成诗+相似诗）"""
    # 读取文件内容
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
    except UnicodeDecodeError:
        with open(file_path, 'r', encoding='gbk') as f:
            content = f.read()
    
    # 步骤1：按模型分组分割内容（匹配你的文件分隔符：============================================================）
    model_blocks = re.split(r'={50,}', content.strip())
    poem_groups = {
        "BASE": {"generated": "", "similar": ""},
        "SFT": {"generated": "", "similar": ""},
        "MINDFLOW": {"generated": "", "similar": ""}
    }
    
    # 步骤2：逐块提取每个模型的诗歌内容
    for block in model_blocks:
        block = block.strip()
        if not block:
            continue
        
        # 匹配模型名称
        model_name = None
        if "📌 BASE 模型" in block:
            model_name = "BASE"
        elif "📌 SFT 模型" in block:
            model_name = "SFT"
        elif "📌 MINDFLOW 模型" in block:
            model_name = "MINDFLOW"
        
        if not model_name:
            continue
        
        # 提取生成诗歌（匹配：生成诗歌：\n[诗歌内容]\n相似度分数：）
        gen_poem_match = re.search(r'生成诗歌：\n(.*?)\n相似度分数：', block, re.DOTALL)
        if gen_poem_match:
            gen_poem = gen_poem_match.group(1).strip()
            poem_groups[model_name]["generated"] = gen_poem
        
        # 提取最相似训练集诗歌（匹配：最相似训练集诗歌：\n[诗歌内容]）
        sim_poem_match = re.search(r'最相似训练集诗歌：\n(.*?)$', block, re.DOTALL)
        if sim_poem_match:
            sim_poem = sim_poem_match.group(1).strip()
            poem_groups[model_name]["similar"] = sim_poem
    
    # 验证提取结果（修复f-string反斜杠错误）
    print("📝 诗歌提取结果验证：")
    for model, poems in poem_groups.items():
        gen_valid = len(poems["generated"].split('\n')) >= 2  # 短诗至少2行
        sim_valid = len(poems["similar"].split('\n')) >= 2
        print(f"- {model} 模型：生成诗有效={gen_valid}，相似诗有效={sim_valid}")
        if gen_valid and sim_valid:
            gen_lines = len(poems['generated'].split('\n'))
            sim_lines = len(poems['similar'].split('\n'))
            print(f"  ✅ 生成诗行数：{gen_lines}（强制≤8行）")
            print(f"  ✅ 相似诗行数：{sim_lines}")
    
    return poem_groups

def plot_single_radar_chart(model_key, generated_scores, similar_scores, save_path):
    """绘制单张雷达图（生成诗 vs 相似诗）- 简化图例"""
    # 雷达图维度（全英文，对应评分的5个核心维度）
    categories = ['Rhyme', 'Meter', 'Rhythm', 'Vocabulary', 'Structure']
    N = len(categories)
    
    # 计算每个维度的角度（雷达图坐标）
    angles = [n / float(N) * 2 * np.pi for n in range(N)]
    angles += angles[:1]  # 闭合雷达图
    
    # 提取得分并闭合数据
    gen_values = [
        generated_scores['rhyme_score'],
        generated_scores['meter_score'],
        generated_scores['rhythm_score'],
        generated_scores['vocabulary_score'],
        generated_scores['structure_score']
    ]
    sim_values = [
        similar_scores['rhyme_score'],
        similar_scores['meter_score'],
        similar_scores['rhythm_score'],
        similar_scores['vocabulary_score'],
        similar_scores['structure_score']
    ]
    gen_values += gen_values[:1]
    sim_values += sim_values[:1]
    
    # 创建画布（高分辨率）
    fig, ax = plt.subplots(figsize=(12, 10), subplot_kw=dict(polar=True))
    
    # 获取模型配置
    model_config = MODEL_GROUPS[model_key]
    
    # 绘制生成诗（实线+填充）- 图例简化为 "Generated Poem (Finetuned)"
    ax.plot(angles, gen_values, linewidth=3, label='Generated Poem (Finetuned)', color=model_config["color1"])
    ax.fill(angles, gen_values, alpha=0.3, color=model_config["color1"])
    
    # 绘制相似诗（虚线+无填充）- 图例简化为 "Most Similar Poem"
    ax.plot(angles, sim_values, linewidth=3, linestyle='--', label='Most Similar Poem', color=model_config["color2"])
    ax.fill(angles, sim_values, alpha=0.1, color=model_config["color2"])
    
    # 设置雷达图样式
    ax.set_theta_offset(np.pi / 2)  # 旋转角度，让第一个维度在顶部
    ax.set_theta_direction(-1)      # 顺时针显示维度
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(categories, fontsize=14, fontweight='bold')
    
    # 统一刻度（0-100，间隔20）
    ax.set_ylim(0, MAX_SCORE)
    ax.set_yticks([0, 20, 40, 60, 80, 100])
    ax.set_yticklabels([f'{x}' for x in [0, 20, 40, 60, 80, 100]], fontsize=10, alpha=0.8)
    ax.grid(True, alpha=0.3)
    
    # 添加标题和图例
    plt.title(f'{model_config["name"]} - Short Poem Score Comparison', 
              fontsize=20, fontweight='bold', pad=30)
    plt.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1), fontsize=12, frameon=True)
    
    # 保存图片（确保路径存在）
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    print(f"✅ 雷达图已保存：{save_path}")
    plt.close()  # 关闭画布释放内存

# -------------------------- 主执行逻辑 --------------------------
if __name__ == "__main__":
    # 1. 配置文件路径和保存路径（替换为你的实际路径）
    file_path = "/root/autodl-tmp/Pro/poetry_results/similarity_对比报告_all_models_poem_20251227_113335.txt"
    save_base_path = "/root/autodl-tmp/Pro/poem_radar_charts/"
    
    # 2. 检查文件是否存在
    if not os.path.exists(file_path):
        print(f"❌ File not found: {file_path}")
        print(f"Current working directory: {os.getcwd()}")
        exit(1)
    print(f"✅ File found at path: {file_path}\n")
    
    # 3. 初始化评分器（适配短诗）
    scorer = EnglishPoemScorer()
    
    # 4. 精准提取诗歌数据（适配你的文件格式）
    print("📥 开始提取诗歌数据...")
    poem_groups = load_poems_from_file(file_path)
    
    # 5. 逐组评分并绘制雷达图（对应你的要求：1,2一张；3,4一张；5,6一张）
    print("\n📊 开始生成雷达图（短诗适配+简化图例）...")
    for model_key in ["BASE", "SFT", "MINDFLOW"]:
        model_poems = poem_groups[model_key]
        model_config = MODEL_GROUPS[model_key]
        
        # 跳过提取不完整的组
        if not model_poems["generated"] or not model_poems["similar"]:
            print(f"⚠️ 跳过 {model_config['name']}：诗歌数据不完整")
            continue
        
        # 计算生成诗评分
        print(f"\n--- 计算 {model_config['name']} 生成诗评分 ---")
        gen_scores = scorer.overall_score(
            poem_text=model_poems["generated"],
            target_pattern="ABAB",
            target_meter="iambic",
            target_form="short_poem",  # 短诗模式
            debug=True
        )
        
        # 计算相似诗评分
        print(f"\n--- 计算 {model_config['name']} 相似诗评分 ---")
        sim_scores = scorer.overall_score(
            poem_text=model_poems["similar"],
            target_pattern="ABAB",
            target_meter="iambic",
            target_form="short_poem",  # 短诗模式
            debug=True
        )
        
        # 打印评分结果
        print(f"\n📈 {model_config['name']} 评分结果（短诗适配）：")
        print(f"生成诗：")
        print(f"  押韵：{gen_scores['rhyme_score']} | 格律：{gen_scores['meter_score']} | 节奏：{gen_scores['rhythm_score']}")
        print(f"  词汇：{gen_scores['vocabulary_score']} | 结构：{gen_scores['structure_score']} | 综合：{gen_scores['overall_score']}")
        print(f"相似诗：")
        print(f"  押韵：{sim_scores['rhyme_score']} | 格律：{sim_scores['meter_score']} | 节奏：{sim_scores['rhythm_score']}")
        print(f"  词汇：{sim_scores['vocabulary_score']} | 结构：{sim_scores['structure_score']} | 综合：{sim_scores['overall_score']}")
        
        # 绘制并保存雷达图
        save_path = f"{save_base_path}{model_key}_short_poem_radar_chart.png"
        plot_single_radar_chart(model_key, gen_scores, sim_scores, save_path)
    
    print(f"\n🎉 所有短诗雷达图生成完成！")
    print(f"📁 保存路径：{save_base_path}")
    print(f"   - BASE模型：BASE_short_poem_radar_chart.png")
    print(f"   - SFT模型：SFT_short_poem_radar_chart.png")
    print(f"   - MINDFLOW模型：MINDFLOW_short_poem_radar_chart.png")

✅ File found at path: /root/autodl-tmp/Pro/poetry_results/similarity_对比报告_all_models_poem_20251227_113335.txt

📥 开始提取诗歌数据...
📝 诗歌提取结果验证：
- BASE 模型：生成诗有效=True，相似诗有效=True
  ✅ 生成诗行数：8（强制≤8行）
  ✅ 相似诗行数：32
- SFT 模型：生成诗有效=True，相似诗有效=True
  ✅ 生成诗行数：8（强制≤8行）
  ✅ 相似诗行数：33
- MINDFLOW 模型：生成诗有效=True，相似诗有效=False

📊 开始生成雷达图（短诗适配+简化图例）...

--- 计算 BASE Model 生成诗评分 ---
🔍 Rhyme Matches: 3/28 (短诗适配)
🔍 Structure Score: 行数8 → 90.0分（短诗适配）

--- 计算 BASE Model 相似诗评分 ---
🔍 Rhyme Matches: 0/28 (短诗适配)
🔍 Structure Score: 行数8 → 90.0分（短诗适配）

📈 BASE Model 评分结果（短诗适配）：
生成诗：
  押韵：28.57 | 格律：44.72 | 节奏：72.36
  词汇：89.29 | 结构：90.0 | 综合：70.32
相似诗：
  押韵：20.0 | 格律：45.03 | 节奏：67.69
  词汇：56.03 | 结构：90.0 | 综合：61.5
✅ 雷达图已保存：/root/autodl-tmp/Pro/poem_radar_charts/BASE_short_poem_radar_chart.png

--- 计算 SFT Model 生成诗评分 ---
🔍 Rhyme Matches: 1/28 (短诗适配)
🔍 Structure Score: 行数8 → 90.0分（短诗适配）

--- 计算 SFT Model 相似诗评分 ---
🔍 Rhyme Matches: 0/28 (短诗适配)
🔍 Structure Score: 行数8 → 90.0分（短诗适配）

📈 SFT Model 评分结果（短诗适配）：
生成诗：
  押韵：22.86 | 格律：38.91 